In [1]:
## Package imports ##
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn import mixture
import pandas as pd
import geopandas as gpd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from dateutil.relativedelta import relativedelta
import datetime as dt
from scipy.optimize import minimize
from scipy.special import gamma as gamma_func, gammaln, gammaincc, exp1

import json
import os
import pprint
import fiona

from functools import partial
import pyproj
from shapely.geometry import Polygon
import shapely.ops as ops


#Check shapely speedups are enabled
from shapely import speedups
speedups.enabled

#Set geopandas settings
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gpd.io.file.fiona.drvsupport.supported_drivers

{'ARCGEN': 'r',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'rw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r',
 'KML': 'rw'}

## Data ingestion

Ingest data from the International Seismological Centre (ISC)

In [ ]:
### ISC Catalog stepwise search ###
# ObsPy plugin breaks when searching for too many events, so we perform search in steps of 1 year (when using FDSN)

def search_ISC_cat_step(search_params_dict):
    """
    Perform ISC catalog search stepwise (in 1 year long steps)
    search_params_dict - dict containing the following keywords:
    start_time (UTCDateTime or dt.datetime), end_time (UTCDateTime or dt.datetime), min_latitude, max_latitude, 
    min_longitude, max_longitude, min_mag, max_mag, min_depth, max_depth, 
    t_step (for segmeting search, type relativedelta)
    """
    calc_start = dt.datetime.now() # time the function
    start_time = search_params_dict["start_time"]
    end_time = search_params_dict["end_time"]
    min_latitude = search_params_dict["min_latitude"]
    max_latitude = search_params_dict["max_latitude"]
    min_longitude = search_params_dict["min_longitude"]
    max_longitude =  search_params_dict["max_longitude"]
    min_mag = search_params_dict["min_mag"]
    max_mag = search_params_dict["max_mag"]
    min_depth = search_params_dict["min_depth"]
    max_depth = search_params_dict["max_depth"]
    t_step = search_params_dict["t_step"]
    
    # Start search
    t1 = start_time
    t2 = t1 + t_step
    #print('Processing:', t1, t2)
    client = Client("ISC")

    # Initialise catalog
    cat_init = client.get_events(starttime=t1,endtime=t2,
                                  minlatitude=min_latitude,maxlatitude=max_latitude,
                                  minlongitude=min_longitude,maxlongitude=max_longitude,
                                  minmagnitude=min_mag, maxmagnitude=max_mag,
                                  mindepth=min_depth, maxdepth=max_depth, orderby="time-asc",
                                  includeallorigins=True, includeallmagnitudes=True)

    # Set up loop for stepwise search
    t1=t2
    t2+=t_step
    print('Beginning loop', t1, t2)
    cat = cat_init
    while t2 < end_time:
        try:
            #print('Loop Processing', t1, t2)
            catalogue = client.get_events(starttime=t1,endtime=t2,
                                  minlatitude=min_latitude,maxlatitude=max_latitude,
                                  minlongitude=min_longitude,maxlongitude=max_longitude,
                                  minmagnitude=min_mag, maxmagnitude=max_mag,
                                  mindepth=min_depth, maxdepth=max_depth, orderby="time-asc",
                                  includeallorigins=True, includeallmagnitudes=True)
            cat=cat.__add__(catalogue)
            t1=t2
            t2+=t_step
        except:
            import sys
            print("Oops!", sys.exc_info()[0], "occurred for ", t1, " - ", t2)
            print('FDSN Web Search failure - finalising catalog...')
            final_cat = cat
            break

    # Add final time step and add to main catalog    
    assert t1 < end_time    
    try:
        cat1 = client.get_events(starttime=t1,endtime=end_time,
                                  minlatitude=min_latitude,maxlatitude=max_latitude,
                                  minlongitude=min_longitude,maxlongitude=max_longitude,
                                  minmagnitude=min_mag, maxmagnitude=max_mag,
                                  mindepth=min_depth, maxdepth=max_depth, orderby="time-asc",
                                  includeallorigins=True, includeallmagnitudes=True)
        final_cat = cat.__add__(cat1)
        print('Final cat', final_cat)
    except:
        import sys
        print("Reminder:", sys.exc_info()[0], "occurred.")
        print('FDSN Web Search failure - catalog now finalised.')
    
    print('    took', (dt.datetime.now() - calc_start), 'to download the EQ catalog\n')
    return final_cat

In [ ]:
### ISC Catalog stepwise search ###

def search_ISC_cat(search_params_dict):
    """
    Perform ISC catalog search stepwise (in 1 year long steps)
    search_params_dict - dict containing the following keywords:
    start_time (UTCDateTime or dt.datetime), end_time (UTCDateTime or dt.datetime), min_latitude, max_latitude, 
    min_longitude, max_longitude, min_mag, max_mag, min_depth, max_depth, 
    t_step (for segmeting search, type relativedelta)
    """
    calc_start = dt.datetime.now() # time the function
    
    start_time = search_params_dict["start_time"]
    end_time = search_params_dict["end_time"]
    min_latitude = search_params_dict["min_latitude"]
    max_latitude = search_params_dict["max_latitude"]
    min_longitude = search_params_dict["min_longitude"]
    max_longitude =  search_params_dict["max_longitude"]
    min_mag = search_params_dict["min_mag"]
    max_mag = search_params_dict["max_mag"]
    min_depth = search_params_dict["min_depth"]
    max_depth = search_params_dict["max_depth"]
    
    # Start search
    client = Client("ISC")

    # Initialise catalog
    cat = client.get_events(starttime=start_time,endtime=end_time,
                                  minlatitude=min_latitude,maxlatitude=max_latitude,
                                  minlongitude=min_longitude,maxlongitude=max_longitude,
                                  minmagnitude=min_mag, maxmagnitude=max_mag,
                                  mindepth=min_depth, maxdepth=max_depth, orderby="time-asc",
                           includeallorigins=True, includeallmagnitudes=True)
    
    print('    took', (dt.datetime.now() - calc_start), 'to download the EQ catalog\n')
    print(cat)
    return cat

In [ ]:
## CREATE CATALOG DATAFRAME ##
def create_cat_df(cat, outfile_path):
    """
    Create pd dataframe of events from ObsPy catalog object
    final_cat: ObsPy catalog object
    outfile_path: str - filepath for output CSV file with catalog data, e.g. Japan_EQ_data
    """
    # Set destination path
    outfile = outfile_path + '/raw_catalog_data.csv'
    
    # Create empty lists
    year = []
    month = []
    day = []
    hour = []
    minute = []
    second = []
    lat = []
    lon = []
    dep = []
    mag = []
    mauth = []
    time = []

    # Loop over each event in the catalogue
    empty = []
    for event in cat:
        year.append(event.origins[0].time.year)
        month.append(event.origins[0].time.month)
        day.append(event.origins[0].time.day)
        hour.append(event.origins[0].time.hour)
        minute.append(event.origins[0].time.minute)
        second.append(event.origins[0].time.second)
        lat.append(event.origins[0].latitude)
        lon.append(event.origins[0].longitude)
        dep.append(event.origins[0].depth)
        ## Magnitudes
        # The following selection is performed for each event:
        # If an event has a GCMT Mw, we pick that. If an event has Mw, but not GCMT, we use the average of all MW/Mw.
        # If an event has no Mw, we use the average of the ISC and NEIC magnitudes, or in the absence of both, we use
        # either ISC or NEIC. If an event has no ISC or NEIC magnitude, we use mags from local agencies, ignoring mags
        # by MOS, the IDC and BJI
        ISC_mags = [] #temporarily holds all event magnitudes for selection
        NEIC_mags = []
        mags_mw = []
        reg_mags = []
        m_gcmt = 0.0
        M = 0.0
        for mags in event.magnitudes: 
            try:
                if mags.creation_info.author == 'GCMT':
                    m_gcmt = mags.mag
                    break
                elif mags.creation_info.author != 'GCMT' and (('W' in mags.magnitude_type) and ('w' in mags.magnitude_type)):
                    ev_mags_mw.append(mags.mag)
                elif mags.creation_info.author == 'ISC':
                    ISC_mags.append(mags.mag)
                elif mags.creation_info.author == 'NEIC':
                    NEIC_mags.append(mags.mag)
                elif (mags.creation_info.author != 'IDC') and (mags.creation_info.author != 'MOS') and (mags.creation_info.author != 'BJI'):
                    reg_mags.append(mags.mag)
            except ValueError:
                M = np.nan

        if m_gcmt != 0.0:
            mag.append(m_gcmt)
            #mauth.append('GCMT')
        elif (mags_mw != empty and m_gcmt == 0.0):
            #print(ev_mags_mw)
            mag.append(np.mean(np.array(mags_mw)))
            #mauth.append('Mw/Any')
        elif (NEIC_mags != empty) and (ISC_mags != empty) and (mags_mw == empty) and (m_gcmt == 0.0):
            #print('NEIC+ISC Chosen: ', np.mean(np.array([max(ISC_mags), max(NEIC_mags)])))
            mag.append(np.mean(np.array([max(ISC_mags), max(NEIC_mags)])))
            #mauth.append('NEIC/ISC')
        elif (NEIC_mags == empty) and (ISC_mags != empty):
            #print('ISC Chosen: ', np.mean(np.array(ISC_mags)))
            mag.append(np.mean(np.array(ISC_mags)))
            #mauth.append('ISC')
        elif (NEIC_mags != empty) and (ISC_mags == empty):
            #print('NEIC Chosen: ', np.mean(np.array(NEIC_mags)))
            mag.append(np.mean(np.array(ISC_mags)))
            #mauth.append('NEIC')
        elif (NEIC_mags == empty) and (ISC_mags == empty) and (reg_mags != empty):
            #print('Regional mags: ', reg_mags)
            mag.append(np.mean(np.array(reg_mags)))
            #mauth.append('Reg')
        else:
            mag.append(np.nan)
            #mauth.append(np.nan)
            

    # Create the dataframe
    #print(len(year), len(month), len(day), len(hour), len(minute), len(second), len(lat), len(lon), len(dep), len(mag)) 
    data = pd.DataFrame(np.array([year, month, day, hour, minute, second, lat, lon, dep, mag]).T, 
                 columns=["year", "month", "day", "hour", "minute", "second",
                          "lat", "lon", "depth_km", "mag"])
    
    # Apply datetimes
    data = data.dropna() # remove events without a magnitude
    data.drop_duplicates(subset=["year", "month", "day", "hour", "minute", "second",
                          "lat", "lon", "depth_km"], keep='first', inplace=True, ignore_index=True)
    data["time"] = pd.to_datetime(data[['year', 'month', 'day', 'hour', 'minute', 'second']])
    data = data.set_index('time')
    data = data.sort_index()
    data = data.reset_index(drop=True)

    print('Catalog contains {} events'.format(len(data.index)))
    # Save raw data to csv
    #data.to_csv(outfile)
    return data

### South American margin

In [ ]:
# South America - using a min mag of 4.5
SAM_search_params1 = {"start_time": dt.datetime(1970,1,1), "end_time": dt.datetime(2021, 7, 1), 
                         "min_latitude": -47, "max_latitude": 8, "min_longitude": -83, "max_longitude": -60, 
                         "min_mag": 4.5, "max_mag": None, "min_depth": None, "max_depth": None, 
                         "t_step": relativedelta(years=1, months=0, days=0)}

SAM_search_result = search_ISC_cat(SAM_search_params1)
print(SAM_search_result)

In [ ]:
create_cat_df(SAM_search_result, 'SAM_EQ_data')

### Japan earthquake catalogs
We need a stepwise search here (`search_ISC_cat_step`) as there too many events in the catalog for a single search, due to the lower magnitude cutoff used. Note that `search_ISC_cat_step` is a lot slower than `search_ISC_cat`.

In [ ]:
# Japan South catalog
Japan_S_search_params1 = {"start_time": dt.datetime(1970,1,1), "end_time": dt.datetime(2021, 7, 1), 
                         "min_latitude": 30, "max_latitude": 36, "min_longitude": 128, "max_longitude": 145, 
                         "min_mag": 3.0, "max_mag": None, "min_depth": None, "max_depth": None, 
                         "t_step": relativedelta(years=1, months=0, days=0)}

Japan_S_search_result = search_ISC_cat_step(Japan_S_search_params1)

In [ ]:
# Japan central
Japan_C_search_params1 = {"start_time": dt.datetime(1970,1,1), "end_time": dt.datetime(2021, 7, 1), 
                         "min_latitude": 36, "max_latitude": 38, "min_longitude": 135, "max_longitude": 147, 
                         "min_mag": 3.0, "max_mag": None, "min_depth": None, "max_depth": None,
                        "t_step": relativedelta(years=1, months=0, days=0)}

Japan_C_search_result = search_ISC_cat_step(Japan_C_search_params1)

In [ ]:
# Japan North
Japan_N_search_params1 = {"start_time": dt.datetime(1970,1,1), "end_time": dt.datetime(2021, 7, 1), 
                         "min_latitude": 38, "max_latitude": 45, "min_longitude": 136, "max_longitude": 152, 
                         "min_mag": 3.0, "max_mag": None, "min_depth": None, "max_depth": None,
                        "t_step": relativedelta(years=1, months=0, days=0)}

Japan_N_search_result = search_ISC_cat_step(Japan_N_search_params1)

In [ ]:
Japan_search_result = Japan_S_search_result.__add__(Japan_C_search_result)
Japan_search_result = Japan_search_result.__add__(Japan_N_search_result)
print(Japan_search_result)

In [ ]:
create_cat_df(Japan_search_result, 'Japan_EQ_data')

### Aleutians

### Indonesia

## Epidemic Type Aftershock Sequence (ETAS) Declustering

### ETAS Implementation by Marsan et al. (2017)
*Please note that this section is still not fully operational - implementation details in Marsan et al. (2017)/Jara et al. (2017) are lacking*

Omori Law parameters are fixed as follows:
$$ \alpha = 2, p = 1, c = 10^{-3} days, \gamma = 2 $$ <br>

Algorithm can be described as follows: <br>
1. Compute triggering rate from catalog as $\nu(x_i,y_i,t_i)/K $ <br>
2. Compute background rate ($ \mu(x_i, y_i, t_i) $), under the assumption of $\omega_i = 0.5$
3. Compute initial total rate as $ \lambda(x_i,y_i,t_i) = \mu(x_i, y_i, t_i) + \nu(x_i, y_i, t_i) $
4. Compute $\omega_i = \frac{\mu(x_i, y_i, t_i)}{\lambda(x_i,y_i,t_i)}$
5. Use $\omega_i $ to compute ML estimate of K

### Equations
$$ \lambda(x,y,t) = \mu(x,y,t) + \nu(x,y,t) $$ <br>
where $\lambda(x,y,t)$ is the total seismicity rate, $\mu(x,y,t)$ is the background seismicity rate, and $\nu(x,y,t)$ is the triggering rate <br>

The triggering rate: <br>
$$ \nu(x,y,t) = \displaystyle\sum_{i/t_i < t}^{} \frac{Ke^{\alpha m_i}}{(t+c-t_i)} \times \frac{\gamma - 1}{2\pi} \times \frac{L_i^{\gamma-1}}{\left((x-x_i)^2 + (y-y_i)^2 + L_i^2 \right )^{\frac{\gamma + 1}{2}}} $$ <br>

The background rate: <br>
$$ \mu(x,y,t) = \displaystyle\sum_{i}^{} \omega_i e^{-\sqrt{(x-x_i)^2 + (y-y_i)^2}/\ell} e^{-|t-t_i|/\tau} \times \frac{1}{2 \pi \ell^2 a_i} $$ <br>

$$ a_i = 2\tau - \tau \left( e^{-\frac{t_s - t_i}{\tau}} - e^{\frac{t_s - t_i}{\tau}} \right) $$ <br>
where $t_s, t_e$ are the start and end times of the catalog

In [ ]:
# Prepare the catalog for processing
def prep_cat_ETAS(cat_init, cat_start, cat_end): 
    """ Loads and prepares the catalog for further processing
    # input cat_init needs to be a file path to a CSV document containing labelled columns:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    """
    # Load catalog from file:
    cat = pd.read_csv(cat_init, index_col=0)
    
    # Apply datetimes
    cat["time"] = pd.to_datetime(cat[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    cat = cat.infer_objects()
    cat.dtypes
    cat.loc[:, 'depth_km'] *=0.001

    # Define a geodataframe using the EQ catalog from above
    cat_gdf = gpd.GeoDataFrame(cat, geometry=gpd.points_from_xy(cat.lon, cat.lat))
    cat_gdf = cat_gdf.set_crs("EPSG:4326")
    # translate target lat, lon to radians for spherical distance calculation
    cat_gdf['lat_rad'] = np.radians(cat_gdf['lat'])
    cat_gdf['lon_rad'] = np.radians(cat_gdf['lon'])
    # Compute the time difference between event occurrence times and the start and end times of the catalog, in days
    cat_gdf['t_diff_e'] = (1./(24.*60.*60.))*((cat_gdf['time']- cat_end).dt.total_seconds())
    cat_gdf['t_diff_s'] = (1./(24.*60.*60.))*((cat_gdf['time'] - cat_start).dt.total_seconds())
    return cat_gdf

In [ ]:
# Haversine formula for computing spherical distances
def hav(theta):
    """Haversine function
    Takes in arguments in radians
    """
    return np.square(np.sin(theta / 2))

def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    #Haversine distance in km - calculate distance between 2 pts on a sphere
    # lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2 must all be in radians
    ####################################################################
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(
        np.sqrt(
            hav(lat_rad_1 - lat_rad_2)
            + np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d

In [ ]:
# Characteristic length/rupture radius in km
def L_i(m, L_0):
    return L_0*np.power(10, 0.5*(m-2))

# a coefficients
# Need to fix abs values in exponents
def a_coeff(t_diff_e,t_diff_s,tau): # tau is the temporal smoothing param; t_diff_e=t_catend-tevent; t_diff_s=t_catstart-tevent
    a = 2*tau - tau*(np.exp(-(np.abs(t_diff_s)/(tau))) - np.exp(-(np.abs(t_diff_e)/(tau)))) # times in days
    return a

# Calculate triggering rate
def nu_calc(c, alpha, p, gamma, K, m, time_diffs, r_sq, L_0):
    # Numerical calculations for nu
    T1 = K*np.exp(alpha*m)/np.power((time_diffs), p)
    #T1 = np.exp(alpha*m)
    T2 = (gamma-1)/2*np.pi
    T3 = np.power(L_i(m, L_0), (gamma-1))
    T4 = np.power((r_sq + np.power(L_i(m, L_0),2)), (gamma+1)/2)
    return T1*T2*(T3/T4)
    
# Calculate the background rate:
def mu_calc(r_sq, t_diff, omega, tau, l, a_coeffs): # tau, l are the temporal and spatial smoothing params
    T1 = omega*np.exp(-np.sqrt(r_sq)/l)
    T2 = np.exp(-np.abs(t_diff)/tau) # times need to be in days
    T3 = 1/(2*np.pi*(l**2)*a_coeffs)
    lam = T1*T2*T3
    return lam

# Triggering rate from catalog
def nuK(catalog, c, alpha, p, gamma, K, L_0):
    """ Calculates nu(xi,yi,ti) - triggering rate at time and place of each event in catalog
    # L_0 is the reference length
    # If K=1, then function actually returns nu(xi,yi,ti)/K
    """
    func_start = dt.datetime.now() # time the function
    cat = catalog.copy(deep=True)
    cat['nuK'] = 0.0
    #print('Looping through catalog...')
    for triggered in cat.itertuples():
        # get values of source event
        ttime = triggered.time
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        potential_triggers = cat.loc[cat["time"] < ttime]
        potential_triggers['c'] = c
        potential_triggers['t_diffs'] = (1./(24.*60.*60.))*(ttime - potential_triggers['time']).dt.total_seconds()
        potential_triggers['t_denom'] = potential_triggers['t_diffs'] + potential_triggers['c']
        #print(potential_triggers['t_denom'])
        #print(potential_triggers)
        # Calculate distances between triggered and potential triggers
        potential_triggers['r_squared'] = np.square(haversine(tlatrad,potential_triggers['lat_rad'],tlonrad,potential_triggers['lon_rad']))
        #print(len(potential_triggers['mag']), len(potential_triggers['r_squared']), len(potential_triggers['t_denom']))
        # Calculate triggering rate nu for each event i.e. nu(xi,yi,ti)
        nuK_array = nu_calc(c, alpha, p, gamma, K, potential_triggers['mag'], potential_triggers['t_denom'], potential_triggers['r_squared'], L_0)
        cat.loc[triggered.Index, 'nuK'] = nuK_array.sum()
    print('    took', (dt.datetime.now() - func_start), 'to compute nu(xi,yi,ti)/K \n')
    return cat

# Background rate from catalog
def mu(catalog, tau, l):
    cat = catalog.copy(deep=True)
    cat['mu_i'] = 0.0
    #cat['omega_i'] = 0.5
    for event in cat.itertuples():
        temp_cat = cat.copy(deep=True)
        evtime = event.time
        #print(evtime)
        #print('Triggered event time:', ttime)
        evlatrad = event.lat_rad
        evlonrad = event.lon_rad
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((evtime - temp_cat['time']).dt.total_seconds())
        
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(evlatrad,temp_cat['lat_rad'],evlonrad,temp_cat['lon_rad']))
        temp_cat['a_coeffs'] = a_coeff(temp_cat['t_diff_e'], temp_cat['t_diff_s'], tau)
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_i'],tau,l,temp_cat['a_coeffs'])
        #print(temp_cat)
        cat.loc[event.Index, 'mu_i'] = temp_cat['mu_indiv'].sum()
    return cat

In [ ]:
# Calculate parameter F_i for individual events
def F_i(alpha, t_diff, c, m):
    return np.exp(alpha*m)*(np.log(t_diff) - np.log(c))

In [ ]:
## Declustering function ##
def decluster(path, cat_start, cat_end, tau, l, c, alpha, p, gamma, L_0, atol):
    """
    # Function to estimate normalisation parameter K and best estimates for omega
    # path must be a filepath (str) to a CSV file containing an output ISC search result with cols:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    # tau, l are temporal and spatial smoothing params, to be given in days and km
    # c, alpha, gamma are Omori-Utsu Law and power spectral density constants
    # L_0 is the reference rupture length
    # atol is the tolerance level for convergence in the MLE estimate of K
    """
    calc_start = dt.datetime.now() # time the function
    assert cat_start < cat_end # Catalog start time must be earlier than the catalog end time
    
    print('\nCatalog start time: ', cat_start, ' Catalog end time:', cat_end, 
          '\nNumber of events in catalog: ', len(cat.index),
          '\nSmoothing time:', tau, 'days '
          'Smoothing length: ', l, 'km ', '\nOmori Law constants: \nc:',  c, 'days', ' alpha: ', alpha, ' p: ', 
          p, ' gamma:', gamma, '\nReference rupture length:', L_0, 'km')
    
    # Load catalog from file and prepare for processing
    print('Preparing catalog for processing...')
    cat_preprocessed = prep_cat_ETAS(path, cat_start, cat_end)
    
    print('Now processing catalog...')
    # Now calculate nu(xi,yi,ti)/K for all events:
    print('Calculating initial triggering rate...')
    nuK_cat = nuK(cat_preprocessed, c, alpha, p, gamma, 1.0, L_0)
    
    nuK_cat['omega_i'] = 0.5
    
    # Calculate background rates at time and place of each event, assuming omega is 0.5
    print('Estimating a priori background rates...')
    initial_mu_cat = mu(nuK_cat, tau, l)
    
    # initial_mu_cat should now contain both a nu and a mu for each event
    # Now compute updated omega:
    initial_mu_cat['lambda_i'] = initial_mu_cat['mu_i'] + initial_mu_cat['nuK']
    initial_mu_cat['omega_i'] = initial_mu_cat['mu_i'] / initial_mu_cat['nuK']
    
    # Initialise some columns for the MLE estimate
    initial_mu_cat['c'] = c # in days
    initial_mu_cat['t_diffs'] = (1./(24.*60.*60.))*(cat_end - initial_mu_cat['time']).dt.total_seconds() # in days
    initial_mu_cat['t_quantity'] = initial_mu_cat['t_diffs'] + initial_mu_cat['c']
    initial_mu_cat['K_num'] = 1 - (initial_mu_cat['omega_i'])
    initial_mu_cat['F_i'] = F_i(alpha, initial_mu_cat['t_quantity'], c, initial_mu_cat['mag'])
    
    # Initialise K - an initial guess
    K = initial_mu_cat['K_num'].sum() / initial_mu_cat['F_i'].sum()
    fevals = 0 # record number of function evaluations so we can later compare methods
    K_prev = K + 2*atol # initialise the previous K simply so that while loop argument is initially true
    updated_cat = initial_mu_cat.copy(deep=True)
    print('Starting iteration for MLE estimate of K')
    while abs(K - K_prev) > atol:
        K_prev = K
        
        # Compute updated nu based on new value of K
        updated_cat = nuK(updated_cat, c, alpha, p, gamma, K, L_0)
        
        # Now compute updated omega:
        updated_cat['lambda_i'] = updated_cat['mu_i'] + updated_cat['nuK']
        updated_cat['omega_i'] = updated_cat['mu_i'] / updated_cat['nuK']
        updated_cat = mu(updated_cat, tau, l)
        updated_cat['K_num'] = 1 - (updated_cat['omega_i'])
        
        # Compute K using updated omega:
        K =  updated_cat['K_num'].sum() / updated_cat['F_i'].sum()
        fevals += 1
        #print('Current iteration solution: ',K)
    print('The final value of K is', K)
    print('\n', fevals, 'function evaluations were required for K convergence')
    
    # Using final K value calculate a final triggering rate nu(xi,yi,ti):
    final_cat = nuK(updated_cat, c, alpha, p, gamma, K, L_0)
    
    # now the final catalog should contain the correct omegas, which can be used to estimate a background seismicity rate curve
    # Check this visually using a histogram
    plt.hist(final_cat["omega_i"],log=True)
    print('    took', (dt.datetime.now() - calc_start), 'for declustering \n')
    return final_cat

In [ ]:
### TESTING ###
cat_start = dt.datetime(1970,1,1)
cat_end = dt.datetime(2021, 6, 21)
cat = catalog_gdf
tau=100; l=100; c=0.001; alpha=2; p=1; gamma=2; L_0=1.78; atol=0.01
print('\nCatalog start time: ', cat_start, ' Catalog end time:', cat_end, 
          '\nNumber of events in catalog: ', len(cat.index),
          '\nSmoothing time:', tau, 'days '
          'Smoothing length: ', l, 'km ', '\nOmori Law constants: \nc:',  c, 'days', ' alpha: ', alpha, ' p: ', 
          p, ' gamma:', gamma, '\nReference rupture length:', L_0, 'km')

# Preprocess
cat_preprocessed = prep_catalog(cat, cat_start, cat_end)

In [ ]:
## Final estimate of background seismicity rate ##
def mu_final(x,y,cat_start, cat_end, cat, tau, l):
    #########################################################
    # Function computes timeseries of the background seismicity rate
    # x,y refer to a spatial reference point - should be the centroid of the study area
    # Function will build an array of datetime objects with a timestep of 1 day using the cat_start, cat_end times
    # catalog should contain omega, a_coeff values for each event - the output of func decluster
    #########################################################
    assert t_cat_start < t_cat_end # Catalog start time must be earlier than the catalog end time
    
    # Time steps for calculating background rate
    times = np.arange(start_time, end_time, dt.timedelta(days=1)).astype(datetime)
    mu_t_series = []
    
    # Compute background rate at each time step
    for t_step in times:
        temp_cat = cat.copy(deep=True)
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((t_step - temp_cat['time']).dt.total_seconds())
        
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(x, temp_cat['lat_rad'], y, temp_cat['lon_rad']))
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_i'],tau,l,temp_cat['a_coeffs'])
        mu_t_series.append(temp_cat['mu_indiv'].sum())
    return times, mu_t_series

#### Declustering implementation

In [ ]:
# ISC web search params
start_time = dt.datetime(1970,1,1)
end_time = dt.datetime(2021, 6, 21)
min_latitude = -25
max_latitude = -11
min_longitude = -80
max_longitude = -66
min_mag = 4.5
max_mag = None
min_depth = None
max_depth = None
print(start_time, end_time)

In [ ]:
### DECLUSTERING ###

# Separate catalog into deep and shallow following Jara et al. (2017)
catalog_shallow = catalog_gdf.loc[catalog_gdf['depth_km'] < 40.0]
catalog_deep = catalog_gdf.loc[catalog_gdf['depth_km'] > 80.0]
declustered_shallow_cat = decluster(catalog_shallow, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)
declustered_deep_cat = decluster(catalog_deep, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)

# And calculate rates:
times, deep_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_deep_cat, tau=100, l=100)
times, shallow_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_shallow_cat, tau=100, l=100)

### ETAS implementation by Mizrahi et al. (2021)

This ETAS implementation by Mizrahi assumes that the background seismicity rate does not vary in time/space. While this may well be valid when attempting to decluster small catalogs (e.g. $10^\circ \times 10^\circ$ area), the assumption almost certainly breaks down when trying to decluster a much larger catalog (e.g. on the scale of an entire margin, like the whole of the South American margin).

In [ ]:
## First find the best-fitting completeness magnitude and the best-fitting b-value
## Using code by Mizrahi et al. (2021) as sourced from https://github.com/lmizrahi/etas/blob/main/mc_b_est.py


##############################################################################
# joint beta and completeness magnitude estimation
# using p-value of Kolmogorov-Smirnov distance to fitted Gutenberg-Richter law
#
# as described by Mizrahi et al., 2021
# Leila Mizrahi, Shyam Nandan, Stefan Wiemer;
# The Effect of Declustering on the Size Distribution of Mainshocks.
# Seismological Research Letters 2021; doi: https://doi.org/10.1785/0220200231
# inspired by method of Clauset et al., 2009
##############################################################################

# mc is the binned completeness magnitude,
# so the 'true' completeness magnitude is mc - delta_m / 2


def round_half_up(n, decimals=0):
    # this is because numpy does weird rounding.
    multiplier = 10 ** decimals
    return np.floor(n * multiplier + 0.5) / multiplier


def estimate_beta_tinti(magnitudes, mc, weights=None, axis=None, delta_m=0):
    # Tinti, S., & Mulargia, F. (1987). Confidence intervals of b values for grouped magnitudes.
    # Bulletin of the Seismological Society of America, 77(6), 2125-2134.

    if delta_m > 0:
        p = (1 + (delta_m / (np.average(magnitudes - mc, weights=weights, axis=axis))))
        beta = 1 / delta_m * np.log(p)
    else:
        beta = 1 / np.average((magnitudes - (mc - delta_m / 2)), weights=weights, axis=axis)
    return beta


def simulate_magnitudes(n, beta, mc):
    mags = np.random.uniform(size=n)
    mags = (-1 * np.log(1 - mags) / beta) + mc
    return mags


def fitted_cdf_discrete(sample, mc, delta_m, x_max=None, beta=None):
    if beta is None:
        beta = estimate_beta_tinti(sample, mc=mc, delta_m=delta_m)

    if x_max is None:
        sample_bin_n = (sample.max() - mc) / delta_m
    else:
        sample_bin_n = (x_max - mc) / delta_m
    bins = np.arange(sample_bin_n + 1)
    cdf = 1 - np.exp(-beta * delta_m * (bins + 1))
    x, y = mc + bins * delta_m, cdf

    x, y_count = np.unique(x, return_counts=True)
    return x, y[np.cumsum(y_count) - 1]


def empirical_cdf(sample, weights=None):
    try:
        sample = sample.values
    except:
        pass
    try:
        weights = weights.values
    except:
        pass

    sample_idxs_sorted = np.argsort(sample)
    sample_sorted = sample[sample_idxs_sorted]
    if weights is not None:
        weights_sorted = weights[sample_idxs_sorted]
        x, y = sample_sorted, np.cumsum(weights_sorted) / weights_sorted.sum()
    else:
        x, y = sample_sorted, np.arange(1, len(sample) + 1) / len(sample)

    # only return one value per bin
    x, y_count = np.unique(x, return_counts=True)
    return x, y[np.cumsum(y_count) - 1]


def ks_test_gr(sample, mc, delta_m, ks_ds=None, n_samples=10000, beta=None):
    sample = sample[sample >= mc - delta_m / 2]
    if len(sample) == 0:
        print("no sample")
        return 1, 0, []
    if len(np.unique(sample)) == 1:
        print("sample contains only one value")
        return 1, 0, []
    if beta is None:
        beta = estimate_beta_tinti(sample, mc=mc, delta_m=delta_m)

    if ks_ds is None:
        ks_ds = []

        n_sample = len(sample)
        simulated_all = round_half_up(
            simulate_magnitudes(mc=mc - delta_m / 2, beta=beta, n=n_samples * n_sample) / delta_m
        ) * delta_m

        x_max = np.max(simulated_all)
        x_fit, y_fit = fitted_cdf_discrete(sample, mc=mc, delta_m=delta_m, x_max=x_max, beta=beta)

        for i in range(n_samples):
            simulated = simulated_all[n_sample * i:n_sample * (i + 1)].copy()
            x_emp, y_emp = empirical_cdf(simulated)
            y_fit_int = np.interp(x_emp, x_fit, y_fit)

            ks_d = np.max(np.abs(y_emp - y_fit_int))
            ks_ds.append(ks_d)
    else:
        x_fit, y_fit = fitted_cdf_discrete(sample, mc=mc, delta_m=delta_m, beta=beta)

    x_emp, y_emp = empirical_cdf(sample)
    y_emp_int = np.interp(x_fit, x_emp, y_emp)

    orig_ks_d = np.max(np.abs(y_fit - y_emp_int))

    return orig_ks_d, sum(ks_ds >= orig_ks_d) / len(ks_ds), ks_ds


def estimate_mc(sample, mcs_test, delta_m, p_pass, stop_when_passed=True, verbose=False, beta=None,
                n_samples=10000):
    """
    sample: np array of magnitudes to test
    mcs_test: completeness magnitudes to test
    delta_m: magnitude bins (sample has to be rounded to bins beforehand)
    p_pass: p-value with which the test is passed
    stop_when_passed: stop calculations when first mc passes the test
    verbose: verbose
    beta: if beta is 'known', only estimate mc
    n_samples: number of magnitude samples to be generated in p-value calculation of KS distance
    """

    ks_ds = []
    ps = []
    i = 0
    for mc in mcs_test:
        if verbose:
            print('\ntesting mc', mc)
        ks_d, p, _ = ks_test_gr(sample, mc=mc, delta_m=delta_m, n_samples=n_samples, beta=beta)

        ks_ds.append(ks_d)
        ps.append(p)

        i += 1
        if verbose:
            print('..p-value: ', p)

        if p >= p_pass and stop_when_passed:
            break
    ps = np.array(ps)
    if np.any(ps >= p_pass):
        best_mc = mcs_test[np.argmax(ps >= p_pass)]
        if beta is None:
            beta = estimate_beta_tinti(sample[sample >= best_mc - delta_m / 2], mc=best_mc, delta_m=delta_m)
        if verbose:
            print("\n\nFirst mc to pass the test:", best_mc, "\nwith a b-value of:", beta/np.log(10))
    else:
        best_mc = None
        beta = None
        if verbose:
            print("None of the mcs passed the test.")

    # beta is the Tinti beta - so b value is beta/ln10
    # return the b-value
    return mcs_test, ks_ds, ps, best_mc, beta/np.log(10)

In [ ]:
##############################################################################
# inversion of ETAS parameters
#
# as described by Mizrahi et al., 2021
# Leila Mizrahi, Shyam Nandan, Stefan Wiemer;
# The Effect of Declustering on the Size Distribution of Mainshocks.
# Seismological Research Letters 2021; doi: https://doi.org/10.1785/0220200231
##############################################################################


# Extract source params from the Coppersmith and Wells empirical relationships
def coppersmith(mag, typ):
    # result is in km

    # typ is one of the following:
    # 1: strike slip fault
    # 2: reverse fault
    # 3: normal fault
    # 4: oblique fault

    if typ == 1:
        # surface rupture length
        SRL = np.power(10, (0.74 * mag - 3.55))
        # subsurface rupture length
        SSRL = np.power(10, (0.62 * mag - 2.57))
        # rupture width
        RW = np.power(10, (0.27 * mag - 0.76))
        # rupture area
        RA = np.power(10, (0.9 * mag - 3.42))
        # average slip
        AD = np.power(10, (0.9 * mag - 6.32))

    elif typ == 2:
        # surface rupture length
        SRL = np.power(10, (0.63 * mag - 2.86))
        # subsurface rupture length
        SSRL = np.power(10, (0.58 * mag - 2.42))
        # rupture width
        RW = np.power(10, (0.41 * mag - 1.61))
        # rupture area
        RA = np.power(10, (0.98 * mag - 3.99))
        # average slip
        AD = np.power(10, (0.08 * mag - 0.74))

    elif typ == 3:
        # surface rupture length
        SRL = np.power(10, (0.5 * mag - 2.01))
        # subsurface rupture length
        SSRL = np.power(10, (0.5 * mag - 1.88))
        # rupture width
        RW = np.power(10, (0.35 * mag - 1.14))
        # rupture area
        RA = np.power(10, (0.82 * mag - 2.87))
        # average slip
        AD = np.power(10, (0.63 * mag - 4.45))

    elif typ == 4:
        # surface rupture length
        SRL = np.power(10, (0.69 * mag - 3.22))
        # subsurface rupture length
        SSRL = np.power(10, (0.59 * mag - 2.44))
        # rupture width
        RW = np.power(10, (0.32 * mag - 1.01))
        # rupture area
        RA = np.power(10, (0.91 * mag - 3.49))
        # average slip
        AD = np.power(10, (0.69 * mag - 4.80))

    return {
        'SRL': SRL,
        'SSRL': SSRL,
        'RW': RW,
        'RA': RA,
        'AD': AD
    }


def rectangle_surface(lat1, lat2, lon1, lon2):
    l = [[lat1, lon1],
         [lat2, lon1],
         [lat2, lon2],
         [lat1, lon2]]
    polygon = Polygon(l)

    geom_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj('EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat1=polygon.bounds[0],
                lat2=polygon.bounds[2])),
        polygon)
    return geom_area.area / 1e6


def polygon_surface(polygon):
    geom_area = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj('EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=polygon.bounds[0],
                lat_2=polygon.bounds[2])),
        polygon)
    return geom_area.area / 1e6


def hav(theta):
    return np.square(np.sin(theta / 2))


def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(
        np.sqrt(
            hav(lat_rad_1 - lat_rad_2)
            + np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)
        )
    )
    return d


def branching_ratio(theta, beta):
    log10_mu, log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    d = np.power(10, log10_d)
    tau = np.power(10, log10_tau)

    eta = beta * k0 * np.pi * np.power(d, -rho) * np.power(tau, -omega) * np.exp(c / tau) * upper_gamma_ext(-omega,c / tau) / (rho * (-a + beta + gamma * rho))
    return eta


def to_days(timediff):
    return timediff / dt.timedelta(days=1)


def upper_gamma_ext(a, x):
    if a > 0:
        return gammaincc(a, x) * gamma_func(a)
    elif a == 0:
        return exp1(x)
    else:
        return (upper_gamma_ext(a + 1, x) - np.power(x, a)*np.exp(-x)) / a


def parameter_array2dict(theta):
    return dict(zip(
        ['log10_mu', 'log10_k0', 'a', 'log10_c', 'omega', 'log10_tau', 'log10_d', 'gamma', 'rho'],
        theta
    ))


def round_nearest_int(num):
    if num-0.5 < np.floor(num):
        num = np.floor(num)
    else:
        num = np.ceil(num)
    return num

def parameter_dict2array(parameters):
    order = ['log10_mu', 'log10_k0', 'a', 'log10_c', 'omega', 'log10_tau', 'log10_d', 'gamma', 'rho']
    return np.array([
        parameters[key] for key in order
    ])


def set_initial_values(ranges=None):
    if ranges is None:
        log10_mu_range = (-10, 0)
        log10_k0_range = (-4, 0)
        a_range = (0.01, 5.)
        log10_c_range = (-8, 0)
        omega_range = (-0.99, 1)
        log10_tau_range = (0.01, 5)
        log10_d_range = (-4, 1)
        gamma_range = (0.01, 5.)
        rho_range = (0.01, 5.)
    else:
        log10_mu_range, log10_k0_range, a_range, log10_c_range, omega_range, log10_tau_range, log10_d_range, gamma_range, rho_range = ranges

    log10_mu = np.random.uniform(*log10_mu_range)
    log10_k0 = np.random.uniform(*log10_k0_range)
    a = np.random.uniform(*a_range)
    log10_c = np.random.uniform(*log10_c_range)
    omega = np.random.uniform(*omega_range)
    log10_tau = np.random.uniform(*log10_tau_range)
    log10_d = np.random.uniform(*log10_d_range)
    gamma = np.random.uniform(*gamma_range)
    rho = np.random.uniform(*rho_range)

    return [
        log10_mu,
        log10_k0,
        a,
        log10_c,
        omega,
        log10_tau,
        log10_d,
        gamma,
        rho
    ]


def prepare_catalog(data, mc, coppersmith_multiplier, timewindow_start, timewindow_end, earth_radius,
                    delta_m=0):
    # precalculates distances in time and space between events that are potentially related to each other
    calc_start = dt.datetime.now()

    # only use data above completeness magnitude
    if delta_m > 0:
        data["mag"] = round_half_up(data["mag"] / delta_m) * delta_m
    relevant = data.query("mag >= @mc").copy()
    relevant.sort_values(by='time', inplace=True)

    # all entries can be sources, but targets only after timewindow start
    targets = relevant.query("time>=@timewindow_start").copy()

    # calculate some source stuff - always assume oblique fault to avoid having to know fault type
    relevant["distance_range_squared"] = np.square(
        coppersmith(relevant["mag"], 4)["SSRL"] * coppersmith_multiplier
    )
    relevant["source_to_end_time_distance"] = to_days(timewindow_end - relevant["time"])
    relevant["pos_source_to_start_time_distance"] = to_days(timewindow_start - relevant["time"]).apply(
        lambda x: max(x, 0)
    )

    # translate target lat, lon to radians for spherical distance calculation
    targets['target_lat_rad'] = np.radians(targets['lat'])
    targets['target_lon_rad'] = np.radians(targets['lon'])
    targets["target_time"] = targets["time"]
    targets["target_id"] = targets.index
    targets["target_time"] = targets["time"]
    # columns that are needed later
    targets["source_id"] = 'i'
    targets["source_magnitude"] = 0.0
    targets["time_distance"] = 0.0
    targets["spatial_distance_squared"] = 0.0
    targets["source_to_end_time_distance"] = 0.0
    targets["pos_source_to_start_time_distance"] = 0.0

    targets = targets.sort_values(by="time")

    # define index and columns that are later going to be needed
    if pd.__version__ >= '0.24.0':
        index = pd.MultiIndex(
            levels=[[], []],
            names=["source_id", "target_id"],
            codes=[[], []]
        )
    else:
        index = pd.MultiIndex(
            levels=[[], []],
            names=["source_id", "target_id"],
            labels=[[], []]
        )

    columns = [
        "target_time",
        "source_magnitude",
        "spatial_distance_squared",
        "time_distance",
        "source_to_end_time_distance",
        "pos_source_to_start_time_distance"
    ]
    res_df = pd.DataFrame(index=index, columns=columns)

    df_list = []

    print('  number of sources:', len(relevant.index))
    print('  number of targets:', len(targets.index))
    for source in relevant.itertuples():
        stime = source.time

        # filter potential targets
        if source.time < timewindow_start:
            potential_targets = targets.copy()
        else:
            potential_targets = targets.query(
                "time>@stime"
            ).copy()
        targets = potential_targets.copy()

        if potential_targets.shape[0] == 0:
            continue

        # get values of source event
        slatrad = np.radians(source.lat)
        slonrad = np.radians(source.lon)
        drs = source.distance_range_squared

        # get source id and info of target events
        potential_targets["source_id"] = source.Index
        potential_targets["source_magnitude"] = source.mag

        # calculate space and time distance from source to target event
        potential_targets["time_distance"] = to_days(potential_targets["target_time"] - stime)

        potential_targets["spatial_distance_squared"] = np.square(
            haversine(
                slatrad,
                potential_targets['target_lat_rad'],
                slonrad,
                potential_targets['target_lon_rad'],
                earth_radius
            )
        )

        # filter for only small enough distances
        potential_targets.query("spatial_distance_squared <= @drs", inplace=True)

        # calculate time distance from source event to timewindow boundaries for integration later
        potential_targets["source_to_end_time_distance"] = source.source_to_end_time_distance
        potential_targets["pos_source_to_start_time_distance"] = source.pos_source_to_start_time_distance

        # append to resulting dataframe
        df_list.append(potential_targets)

    res_df = pd.concat(df_list)[["source_id", "target_id"] + columns].reset_index().set_index(
        ["source_id", "target_id"])

    print('    took', (dt.datetime.now() - calc_start), 'to prepare the distances\n')

    return res_df


def triggering_kernel(metrics, params):
    # metrics are the time-distance, spatial distance squared, and source magnitude
    # params are the ETAS 
    # given time distance in days and squared space distance in square km and magnitude of target event,
    # calculate the (not normalized) likelihood, that source event triggered target event

    time_distance, spatial_distance_squared, m = metrics
    theta, mc = params

    log10_mu, log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta
    mu = np.power(10, log10_mu)
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    tau = np.power(10, log10_tau)
    d = np.power(10, log10_d)

    aftershock_number = k0 * np.exp(a * (m - mc))
    time_decay = np.exp(-time_distance / tau) / np.power((time_distance + c), (1 + omega))
    space_decay = 1 / np.power(
        (spatial_distance_squared + d * np.exp(gamma * (m - mc))),
        (1 + rho)
    )

    res = aftershock_number * time_decay * space_decay
    return res


def expectation_step(distances, target_events, source_events, params, verbose=False):
    calc_start = dt.datetime.now()
    theta, mc = params
    log10_mu, log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta
    # print('I am doing the expectation step with parameters', theta)
    mu = np.power(10, log10_mu)
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    tau = np.power(10, log10_tau)
    d = np.power(10, log10_d)

    # calculate the triggering density values gij
    if verbose:
        print('    calculating gij')
    Pij_0 = distances.copy()
    Pij_0["gij"] = triggering_kernel(
        [
            Pij_0["time_distance"],
            Pij_0["spatial_distance_squared"],
            Pij_0["source_magnitude"]
        ],
        [theta, mc]
    )

    # calculate muj for each target. currently constant, could be improved - this is a problem!
    target_events_0 = target_events.copy()
    target_events_0["mu"] = mu

    # calculate triggering probabilities Pij
    if verbose:
        print('    calculating Pij')
    Pij_0["tot_rates"] = 0
    Pij_0["tot_rates"] = Pij_0["tot_rates"].add(Pij_0["gij"].sum(level=1)).add(target_events_0["mu"])
    Pij_0["Pij"] = Pij_0["gij"].div(Pij_0["tot_rates"])

    # calculate probabilities of being triggered or background
    target_events_0["P_triggered"] = 0
    target_events_0["P_triggered"] = target_events_0["P_triggered"].add(Pij_0["Pij"].sum(level=1)).fillna(0)
    target_events_0["P_background"] = 1 - target_events_0["P_triggered"]

    # calculate expected number of background events
    if verbose:
        print('    calculating n_hat and l_hat\n')
    n_hat_0 = target_events_0["P_background"].sum()

    # calculate aftershocks per source event
    source_events_0 = source_events.copy()
    source_events_0["l_hat"] = Pij_0["Pij"].sum(level=0)

    print('    expectation step took ', dt.datetime.now() - calc_start)
    return Pij_0, target_events_0, source_events_0, n_hat_0


def expected_aftershocks(event, params, no_start=False, no_end=False):
    theta, mc = params

    log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    tau = np.power(10, log10_tau)
    d = np.power(10, log10_d)

    if no_start:
        if no_end:
            event_magnitude = event
        else:
            event_magnitude, event_time_to_end = event
    else:
        if no_end:
            event_magnitude, event_time_to_start = event
        else:
            event_magnitude, event_time_to_start, event_time_to_end = event

    number_factor = k0 * np.exp(a * (event_magnitude - mc))
    area_factor = np.pi * np.power(
        d * np.exp(gamma * (event_magnitude - mc)),
        -1 * rho
    ) / rho

    time_factor = np.exp(c/tau) * np.power(tau, -omega)  # * gamma_func(-omega)

    if no_start:
        time_fraction = upper_gamma_ext(-omega, c/tau)
    else:
        time_fraction = upper_gamma_ext(-omega, (event_time_to_start + c)/tau)
    if not no_end:
        time_fraction = time_fraction - upper_gamma_ext(-omega, (event_time_to_end + c)/tau)

    time_factor = time_factor * time_fraction

    return number_factor * area_factor * time_factor


def ll_aftershock_term(l_hat, g):
    mask = g != 0
    term = -1 * gammaln(l_hat + 1) - g
    term = term + l_hat * np.where(mask, np.log(g, where=mask), -300)
    return term


def neg_log_likelihood(theta, args):
    mc, n_hat, Pij, source_events, timewindow_length, area = args

    assert Pij.index.names == ("source_id", "target_id"), "Pij must have multiindex with names 'source_id', 'target_id'"
    assert source_events.index.name == "source_id", "source_events must have index with name 'source_id'"

    log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    tau = np.power(10, log10_tau)
    d = np.power(10, log10_d)

    source_events["G"] = expected_aftershocks(
        [
            source_events["source_magnitude"],
            source_events["pos_source_to_start_time_distance"],
            source_events["source_to_end_time_distance"]
        ],
        [theta, mc]
    )

    aftershock_term = ll_aftershock_term(
        source_events["l_hat"],
        source_events["G"],
    ).sum()

    # space time distribution term
    Pij["likelihood_term"] = (
        (omega * np.log(tau) - np.log(upper_gamma_ext(-omega, c/tau))
         + np.log(rho) + rho * np.log(
            d * np.exp(gamma * (Pij["source_magnitude"] - mc))
        ))
        - ((1 + rho) * np.log(
            Pij["spatial_distance_squared"] + (
                d * np.exp(gamma * (Pij["source_magnitude"] - mc))
            )
        ))
        - (1 + omega) * np.log(Pij["time_distance"] + c)
        - (Pij["time_distance"] + c) / tau
        - np.log(np.pi)

    )
    distribution_term = Pij["Pij"].mul(Pij["likelihood_term"]).sum()

    total = aftershock_term + distribution_term

    return -1 * total


def optimize_parameters(theta_0, ranges, args):
    start_calc = dt.datetime.now()

    mc, n_hat, Pij, source_events, timewindow_length, area = args
    log10_mu_range, log10_k0_range, a_range, log10_c_range, omega_range, log10_tau_range, log10_d_range, gamma_range, rho_range = ranges

    log10_mu, log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho = theta_0
    mu = np.power(10, log10_mu)
    k0 = np.power(10, log10_k0)
    c = np.power(10, log10_c)
    tau = np.power(10, log10_tau)
    d = np.power(10, log10_d)

    # estimate mu independently and remove from parameters
    mu_hat = n_hat / (area * timewindow_length)
    theta_0_without_mu = log10_k0, a, log10_c, omega, log10_tau, log10_d, gamma, rho

    bounds = [
        log10_k0_range,
        a_range,
        log10_c_range,
        omega_range,
        log10_tau_range,
        log10_d_range,
        gamma_range,
        rho_range
    ]

    res = minimize(
        neg_log_likelihood,
        x0=theta_0_without_mu,
        bounds=bounds,
        args=args,
        tol=1e-12,
    )

    new_theta_without_mu = res.x
    new_theta = [np.log10(mu_hat), *new_theta_without_mu]

    print("    optimization step took ", dt.datetime.now() - start_calc)

    return np.array(new_theta)


def invert_etas_params(
        metadata,
        timewindow_end=None,
        globe=False,
        store_pij=False,
        store_results=True
):
    """
        Inverts ETAS parameters.
        metadata can be either a string (path to json file with stored metadata)
        or a dict. accepted & necessary keywords are:
            fn_catalog: filename of the catalog (absolute path or filename in current directory)
                        catalog is expected to be a csv file with the following columns:
                        id, latitude, longitude, time, magnitude
                        id needs to contain a unique identifier for each event
                        time contains datetime of event occurrence
                        see synthetic_catalog.csv for an example
            data_path: path where result data will be stored
            auxiliary_start: start date of the auxiliary catalog (str or datetime).
                             events of the auxiliary catalog act as sources, not as targets
            timewindow_start: start date of the primary catalog , end date of auxiliary catalog (str or datetime).
                             events of the primary catalog act as sources and as targets
            timewindow_end: end date of the primary catalog (str or datetime)
            mc: cutoff magnitude. catalog needs to be complete above mc
            delta_m: size of magnitude bins
            coppersmith_multiplier: events further apart from each other than
                                    coppersmith subsurface rupture length * this multiplier
                                    are considered to be uncorrelated (to reduce size of distance matrix)
            shape_coords: coordinates of the boundary of the region to consider
                          (list of lists, i.e. [[lat1, lon1], [lat2, lon2], [lat3, lon3]])
    """
    ####################
    # preparing metadata
    ####################
    print("PREPARING METADATA...\n")

    if isinstance(metadata, str):
        # if metadata is a filename, read the file (assuming it's json)
        with open(metadata, 'r') as f:
            parameters_dict = json.load(f)
    else:
        parameters_dict = metadata

    fn_catalog = parameters_dict["fn_catalog"]
    print("  using catalog: " + fn_catalog)

    data_path = parameters_dict["data_path"]
    if data_path == "":
        print("  Data will be stored in " + os.path.dirname(os.path.abspath(__file__)))
    else:
        print("  Data will be stored in " + data_path)

    auxiliary_start = pd.to_datetime(parameters_dict["auxiliary_start"])
    timewindow_start = pd.to_datetime(parameters_dict["timewindow_start"])
    timewindow_end = timewindow_end or pd.to_datetime(parameters_dict["timewindow_end"])
    print(
        "  Time Window: " + str(auxiliary_start)
        + " (aux) - " + str(timewindow_start) + " (start) - " + str(timewindow_end) + " (end)"
    )

    mc = parameters_dict["mc"]
    delta_m = parameters_dict["delta_m"]
    print("  mc is " + str(mc) + " and delta_m is " + str(delta_m))

    coppersmith_multiplier = parameters_dict["coppersmith_multiplier"]
    print("  coppersmith multiplier is " + str(coppersmith_multiplier))

    if globe:
        coordinates = []
    else:
        if type(parameters_dict["shape_coords"]) is str:
            coordinates = np.array(eval(parameters_dict["shape_coords"]))
        else:
            coordinates = np.array(parameters_dict["shape_coords"])
        pprint.pprint("  Coordinates of region: " + str(list(coordinates)))

    # defining some other stuff here..

    timewindow_length = to_days(timewindow_end - timewindow_start)

    fn_parameters = data_path + 'parameters.json'
    fn_ip = data_path + 'ind_and_bg_probs.csv'
    fn_src = data_path + 'sources.csv'
    fn_dist = data_path + 'distances.csv'
    fn_pij = data_path + 'pij.csv'

    # earth radius in km
    earth_radius = 6.3781e3

    if globe:
        area = earth_radius ** 2 * 4 * np.pi
    else:
        poly = Polygon(coordinates)
        area = polygon_surface(poly)
    print("  Region has " + str(area) + " square km")

    # ranges for parameters
    log10_mu_range = (-10, 0)
    log10_k0_range = (-4, 0)
    a_range = (0.01, 5.)
    log10_c_range = (-8, 0)
    omega_range = (-0.99, 1)
    log10_tau_range = (0.01, 5)
    log10_d_range = (-4, 3)
    gamma_range = (0.01, 5.)
    rho_range = (0.01, 5.)

    ranges = log10_mu_range, log10_k0_range, a_range, log10_c_range, omega_range, log10_tau_range, log10_d_range, gamma_range, rho_range

    # start inversion
    print("\n\nINITIALIZING\n")
    print("  reading data..\n")
    df_full = pd.read_csv(fn_catalog, index_col=0, dtype={"url": str, "alert": str})
    df_full["time"] = pd.to_datetime(df_full[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    df_full = df_full.infer_objects()
    #Create gdf
    gdf = gpd.GeoDataFrame(
        df_full, geometry=gpd.points_from_xy(df_full.lat, df_full.lon))

    # filter for events in region of interest
    #if not globe:
        #df = gdf[gdf.within(poly)].copy()
        #df.drop("geometry", axis=1, inplace=True)
    #else:
    df = df_full.copy()

    print("  "+str(len(df)) + " out of " + str(len(df_full)) + " events lie within target region.")

    # filter for events above cutoff magnitude - delta_m/2
    if delta_m > 0:
        df["mag"] = round_half_up(df["mag"] / delta_m) * delta_m
    df.query("mag>=@mc-@delta_m/2", inplace=True)

    print("  "+str(len(df)) + " events are above cutoff magnitude")

    # filter for eventsin relevant timewindow
    df.query("time >= @ auxiliary_start and time < @ timewindow_end", inplace=True)

    print("  "+str(len(df)) + " events are within time window\n\n")

    print('  calculating distances..\n')

    distances = prepare_catalog(
        df,
        mc=mc - delta_m / 2,
        coppersmith_multiplier=coppersmith_multiplier,
        timewindow_start=timewindow_start,
        timewindow_end=timewindow_end,
        earth_radius=earth_radius,
        delta_m=delta_m
    )
    # distances.to_csv(fn_dist)

    print('  preparing source and target events..\n')

    target_events = df.copy()
    target_events.query("time > @ timewindow_start", inplace=True)
    target_events.index.name = "target_id"

    beta = estimate_beta_tinti(target_events["mag"], mc=mc, delta_m=delta_m)
    print("  beta of primary catalog is", beta)

    source_columns = [
        "source_magnitude",
        "source_to_end_time_distance",
        "pos_source_to_start_time_distance"
    ]

    source_events = pd.DataFrame(
        distances[
            source_columns
        ].groupby("source_id").first()
    )

    try:
        print('  using input initial values for theta\n')
        initial_values = parameter_dict2array(
            parameters_dict["theta_0"]
        )
    except KeyError:
        print('  randomly chosing initial values for theta\n')
        initial_values = set_initial_values()


    #################
    # start inversion
    #################
    print('\n\nSTART INVERSION!\n')

    diff_to_before = 100
    i = 0
    while diff_to_before >= 0.001:
        print('iteration ' + str(i) + '\n')

        if i == 0:
            parameters = initial_values

        print('  expectation\n')
        Pij, target_events, source_events, n_hat = expectation_step(
            distances=distances,
            target_events=target_events,
            source_events=source_events,
            params=[parameters, mc - delta_m / 2],
            verbose=True
        )
        print('      n_hat:', n_hat, '\n')

        print('  maximization\n')
        args = [mc - delta_m / 2, n_hat, Pij, source_events, timewindow_length, area]

        new_parameters = optimize_parameters(
            theta_0=parameters,
            args=args,
            ranges=ranges
        )
        print('    new parameters:\n')
        pprint.pprint(
            parameter_array2dict(new_parameters),
            indent=4
        )
        diff_to_before = np.sum(np.abs(parameters - new_parameters))
        print('\n    difference to previous:', diff_to_before)

        br = branching_ratio(parameters, beta)
        print('    branching ratio:', br, '\n')
        parameters = new_parameters
        i += 1

    print('stopping here. converged after', i, 'iterations.')
    print('  last expectation step\n')
    Pij, target_events, source_events, n_hat = expectation_step(
        distances=distances,
        target_events=target_events,
        source_events=source_events,
        params=[parameters, mc - delta_m / 2],
        verbose=True
    )
    print('      n_hat:', n_hat)
    if store_results:
        target_events.to_csv(fn_ip)
        source_events.to_csv(fn_src)

        all_info = {
            "auxiliary_start": str(auxiliary_start),
            "timewindow_start": str(timewindow_start),
            "timewindow_end": str(timewindow_end),
            "timewindow_length": timewindow_length,
            "mc": mc,
            "beta": beta,
            "n_target_events": len(target_events),
            "delta_m": delta_m,
            "shape_coords": str(list(coordinates)),
            "earth_radius": earth_radius,
            "area": area,
            "coppersmith_multiplier": coppersmith_multiplier,
            "log10_mu_range": log10_mu_range,
            "log10_k0_range": log10_k0_range,
            "a_range": a_range,
            "log10_c_range": log10_c_range,
            "omega_range": omega_range,
            "log10_tau_range": log10_tau_range,
            "log10_d_range": log10_d_range,
            "gamma_range": gamma_range,
            "rho_range": rho_range,
            "ranges": ranges,
            "fn": fn_catalog,
            "fn_dist": fn_dist,
            "fn_ip": fn_ip,
            "fn_src": fn_src,
            "calculation_date": str(dt.datetime.now()),
            "initial_values": str(parameter_array2dict(initial_values)),
            "final_parameters": str(parameter_array2dict(new_parameters)),
            "n_iterations": i
        }

        info_json = json.dumps(all_info)
        f = open(fn_parameters, "w")
        f.write(info_json)
        f.close()

    if store_pij:
        Pij.to_csv(fn_pij)

    return parameter_array2dict(new_parameters)

In [ ]:
## Decluster catalog based on the Mizrahi functions
# This is simple, threshold-based declustering based on the Mizrahi ETAS-Background algorithm 
# splits catalog into background and triggered events
    
def Mizrahi_ETAS_BG_decluster(
        metadata,
        timewindow_end=None,
        globe=False,
        store_pij=False,
        store_results=True):
    """
        Declusters an earthquake catalog using the ETAS-based approach of Mizrahi et al. (2021).
        metadata can be either a string (path to json file with stored metadata)
        or a dict. accepted & necessary keywords are:
            fn_catalog: filename of the catalog (absolute path or filename in current directory)
                        catalog is expected to be a csv file with the following columns:
                        id, latitude, longitude, time, magnitude
                        id needs to contain a unique identifier for each event
                        time contains datetime of event occurrence
                        see synthetic_catalog.csv for an example
            data_path: path where result data will be stored
            auxiliary_start: start date of the auxiliary catalog (str or datetime).
                             events of the auxiliary catalog act as sources, not as targets
            timewindow_start: start date of the primary catalog , end date of auxiliary catalog (str or datetime).
                             events of the primary catalog act as sources and as targets
            timewindow_end: end date of the primary catalog (str or datetime)
            mc: cutoff magnitude. catalog needs to be complete above mc
            delta_m: size of magnitude bins
            coppersmith_multiplier: events further apart from each other than
                                    coppersmith subsurface rupture length * this multiplier
                                    are considered to be uncorrelated (to reduce size of distance matrix)
            shape_coords: coordinates of the boundary of the region to consider
                          (list of lists, i.e. [[lat1, lon1], [lat2, lon2], [lat3, lon3]])
    """
    ####################
    # preparing metadata
    ####################
    print("PREPARING METADATA...\n")

    if isinstance(metadata, str):
        # if metadata is a filename, read the file (assuming it's json)
        with open(metadata, 'r') as f:
            parameters_dict = json.load(f)
    else:
        parameters_dict = metadata

    fn_catalog = parameters_dict["fn_catalog"]
    print("  using catalog: " + fn_catalog)

    data_path = parameters_dict["data_path"]
    if data_path == "":
        print("  Data will be stored in " + os.path.dirname(os.path.abspath(__file__)))
    else:
        print("  Data will be stored in " + data_path)

    auxiliary_start = pd.to_datetime(parameters_dict["auxiliary_start"])
    timewindow_start = pd.to_datetime(parameters_dict["timewindow_start"])
    timewindow_end = timewindow_end or pd.to_datetime(parameters_dict["timewindow_end"])
    print(
        "  Time Window: " + str(auxiliary_start)
        + " (aux) - " + str(timewindow_start) + " (start) - " + str(timewindow_end) + " (end)"
    )

    mc = parameters_dict["mc"]
    delta_m = parameters_dict["delta_m"]
    print("  mc is " + str(mc) + " and delta_m is " + str(delta_m))

    coppersmith_multiplier = parameters_dict["coppersmith_multiplier"]
    print("  coppersmith multiplier is " + str(coppersmith_multiplier))

    if globe:
        coordinates = []
    else:
        if type(parameters_dict["shape_coords"]) is str:
            coordinates = np.array(eval(parameters_dict["shape_coords"]))
        else:
            coordinates = np.array(parameters_dict["shape_coords"])
        pprint.pprint("  Coordinates of region: " + str(list(coordinates)))

    # defining some other stuff here..

    timewindow_length = to_days(timewindow_end - timewindow_start)

    fn_parameters = data_path + 'parameters.json'
    fn_ip = data_path + 'ind_and_bg_probs.csv'
    fn_src = data_path + 'sources.csv'
    fn_dist = data_path + 'distances.csv'
    fn_pij = data_path + 'pij.csv'
    fn_cat = data_path + 'ETAS_declustered_cat.csv'
    fn_rejcat = data_path + 'ETAS_rejected_evs.csv'


    # earth radius in km
    earth_radius = 6.3781e3

    if globe:
        area = earth_radius ** 2 * 4 * np.pi
    else:
        poly = Polygon(coordinates)
        area = polygon_surface(poly)
    print("  Region has " + str(area) + " square km")

    # Initialise key data columns
    print("\n\nINITIALIZING\n")
    print("  reading data..\n")
    df_full = pd.read_csv(fn_catalog, index_col=0, dtype={"url": str, "alert": str})
    df_full["time"] = pd.to_datetime(df_full[['year', 'month', 'day', 'hour', 'minute', 'second']])
    df_full['depth_km'] *=0.001


    #Fix dtypes
    df_full = df_full.infer_objects()
    #Create gdf
    gdf = gpd.GeoDataFrame(
        df_full, geometry=gpd.points_from_xy(df_full.lat, df_full.lon))

    df = df_full.copy()

    print("  "+str(len(df)) + " out of " + str(len(df_full)) + " events lie within target region.")

    # filter for events above cutoff magnitude - delta_m/2
    print('Cutoff mag is: ', mc-delta_m/2)
    if delta_m > 0:
        df["mag"] = round_half_up(df["mag"] / delta_m) * delta_m
    m_cut = mc-delta_m/2
    df.query("mag >= @m_cut", inplace=True)

    print("  "+str(len(df)) + " events are above cutoff magnitude")

    # filter for eventsin relevant timewindow
    df.query("time >= @ auxiliary_start and time < @ timewindow_end", inplace=True)

    print("  "+str(len(df)) + " events are within time window\n\n")

    print('  calculating distances..\n')

    distances = prepare_catalog(
        df,
        mc=mc - delta_m / 2,
        coppersmith_multiplier=coppersmith_multiplier,
        timewindow_start=timewindow_start,
        timewindow_end=timewindow_end,
        earth_radius=earth_radius,
        delta_m=delta_m
    )
    # distances.to_csv(fn_dist)

    print('  preparing source and target events..\n')

    target_events = df.copy()
    target_events.query("time > @ timewindow_start", inplace=True)
    target_events.index.name = "target_id"

    beta = estimate_beta_tinti(target_events["mag"], mc=mc, delta_m=delta_m)
    print("  beta of primary catalog is", beta)

    source_columns = [
        "source_magnitude",
        "source_to_end_time_distance",
        "pos_source_to_start_time_distance"
    ]

    source_events = pd.DataFrame(
        distances[
            source_columns
        ].groupby("source_id").first()
    )

    try:
        print('  using input initial values for theta\n')
        initial_values = parameter_dict2array(
            parameters_dict["theta_0"]
        )
    except KeyError:
        print('  randomly chosing initial values for theta\n')
        initial_values = set_initial_values()


    #################################
    # perform probability calculation
    #################################

    parameters = initial_values
    print('  Performing expectation step calculation\n')
    Pij, target_events, source_events, n_hat = expectation_step(
        distances=distances,
        target_events=target_events,
        source_events=source_events,
        params=[parameters, mc - delta_m / 2],
        verbose=True)
    
    print('      n_hat:', n_hat) # n_hat is the no of expected independent events
    
    ###########################
    # perform the declustering
    ###########################
    # Compute threshold probability for declustering
    print('\nPERFORMING "ETAS-Background" DECLUSTERING \n')
    n_est = round_nearest_int(n_hat)
    print('Target number of independent events: {}'.format(n_est))
    
    p_range = np.flip(np.linspace(0.0, 1.0, 101))
    diffs = []
    p_thresh = None
    
    for p_indep in p_range:
        #print('\nTesting {} as threshold probability'.format(p_indep))
        n_E_ind = len(target_events.loc[target_events['P_background'] >= p_indep ].index)
        #print('No of events above p_thresh={}: {}'.format(p_indep, n_E_ind))
        diff = np.abs(n_E_ind - n_est)
        diffs.append(diff)
        if n_E_ind == n_est:
            p_thresh = p_indep
            print('Fitting threshold probability is: ', p_thresh)
            break
            
    if p_thresh is None:
        p_thresh = p_range[diffs.index(min(diffs))]
        print('Fitting threshold probability is: ', p_thresh)
        
    
    declustered_cat = target_events.loc[(target_events['P_background'] >= p_thresh)]
    rejected_ev = target_events.loc[target_events['P_background'] < p_thresh]
    print('{} earthquakes out of {} classified as background events. \n {:.1f}% of total'.format(len(declustered_cat.index), 
                            len(target_events.index), (100*len(declustered_cat.index)/len(target_events.index))))
    
    # Store results to CSV and metadata to json
    if store_results:
        target_events.to_csv(fn_ip)
        source_events.to_csv(fn_src)
        declustered_cat.to_csv(fn_cat)
        rejected_ev.to_csv(fn_rejcat)

        all_info = {
            "auxiliary_start": str(auxiliary_start),
            "timewindow_start": str(timewindow_start),
            "timewindow_end": str(timewindow_end),
            "timewindow_length": timewindow_length,
            "mc": mc,
            "beta": beta,
            "n_target_events": len(target_events),
            "delta_m": delta_m,
            "shape_coords": str(list(coordinates)),
            "earth_radius": earth_radius,
            "area": area,
            "coppersmith_multiplier": coppersmith_multiplier,
            "fn": fn_catalog,
            "fn_dist": fn_dist,
            "fn_ip": fn_ip,
            "fn_src": fn_src,
            "calculation_date": str(dt.datetime.now()),
            "initial_values": str(parameter_array2dict(initial_values)),
        }

        info_json = json.dumps(all_info)
        f = open(fn_parameters, "w")
        f.write(info_json)
        f.close()

    if store_pij:
        Pij.to_csv(fn_pij)
    
    return declustered_cat, rejected_ev

### South America

In [ ]:
# Inversion for the ETAS params
theta_0 = {'log10_mu': -5.8,'log10_k0': -2.6, 'a': 1.8, 'log10_c': -2.5, 'omega': -0.02,
        'log10_tau': 3.5, 'log10_d': -0.85,'gamma': 1.3,'rho': 0.66}

inversion_meta = {"fn_catalog": "SAM_EQ_data/raw_catalog_data.csv","data_path": "SAM_EQ_data/Mizrahi_ETAS_inversion/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1),
        "theta_0": theta_0,"mc": 4.8,"delta_m": 0.1,"coppersmith_multiplier": 100,
"shape_coords": [[-25.0, -66.0], [-25.0, -80.0], [-11.0, -66.0], [-11.0, -80.0],[-25.0, -66.0]],}

parameters = invert_etas_params(inversion_meta,)

In [ ]:
# Define dict of ETAS parameters - based on params recovered by inversion
theta_0 = parameters

# Define metadata
decluster_meta = {"fn_catalog": "SAM_EQ_data/raw_catalog_data.csv","data_path": "SAM_EQ_data/Mizrahi_ETAS_decluster/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1),
        "theta_0": theta_0,"mc": 4.8,"delta_m": 0.1,"coppersmith_multiplier": 100,
"shape_coords": [[-47.0, -83.0], [-47.0, -60.0], [8.0, -60.0], [8.0, -83.0],[-47.0, -83.0]],}

ETAS_declustered_cat, ETAS_rejected_cat = Mizrahi_ETAS_BG_decluster(decluster_meta,)

### Japan

In [ ]:
# Inversion for the ETAS params
theta_0 = {'log10_mu': -5.8,'log10_k0': -2.6, 'a': 1.8, 'log10_c': -2.5, 'omega': -0.02,
        'log10_tau': 3.5, 'log10_d': -0.85,'gamma': 1.3,'rho': 0.66}

inversion_meta = {"fn_catalog": "Japan_EQ_data/raw_catalog_data.csv","data_path":"Japan_EQ_data/Mizrahi_ETAS_inversion/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1),
        "theta_0": theta_0,"mc": 3.0,"delta_m": 0.1,"coppersmith_multiplier": 100,
"shape_coords": [[30.0,128.0],[36.0, 128.0],[36.0, 135.0],[38.0, 135.0],[38.0, 136.0],[45.0, 136.0], 
                 [45.0, 152.0],[38.0, 152.0],[38.0, 147.0],[36.0, 147.0],[36.0, 145.0],[30.0, 145.0],[30.0, 128.0]],}

parameters = invert_etas_params(inversion_meta,)

In [ ]:
# Define dict of ETAS parameters - based on params recovered by inversion
theta_0 = parameters

# Define metadata
decluster_meta = {"fn_catalog": "Japan_EQ_data/raw_catalog_data.csv","data_path": "Japan_EQ_data/Mizrahi_ETAS_decluster/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1),
        "theta_0": theta_0,"mc": 3.0,"delta_m": 0.1,"coppersmith_multiplier": 100,
"shape_coords": [[128.0, 30.0],[128.0, 36.0],[135.0, 36.0],[135.0, 38.0],[136.0, 38.0],[136.0, 45.0], 
                 [152.0, 45.0],[152.0, 38.0],[147.0, 38.0],[147.0, 36.0],[145.0, 36.0],[145.0, 30.0],[128.0, 30.0]],}

ETAS_declustered_cat, ETAS_rejected_cat = Mizrahi_ETAS_BG_decluster(decluster_meta,)

#### Mizrahi-Main

In [ ]:
# Development of Mizrahi method where events get split into clusters - work in progress...
def Mizrahi_ETAS_Main_decluster(
        metadata,
        timewindow_end=None,
        globe=False,
        store_pij=False,
        store_results=True):
    """
        Declusters an earthquake catalog using the ETAS-based approach of Mizrahi et al. (2021).
        metadata can be either a string (path to json file with stored metadata)
        or a dict. accepted & necessary keywords are:
            fn_catalog: filename of the catalog (absolute path or filename in current directory)
                        catalog is expected to be a csv file with the following columns:
                        id, latitude, longitude, time, magnitude
                        id needs to contain a unique identifier for each event
                        time contains datetime of event occurrence
                        see synthetic_catalog.csv for an example
            data_path: path where result data will be stored
            auxiliary_start: start date of the auxiliary catalog (str or datetime).
                             events of the auxiliary catalog act as sources, not as targets
            timewindow_start: start date of the primary catalog , end date of auxiliary catalog (str or datetime).
                             events of the primary catalog act as sources and as targets
            timewindow_end: end date of the primary catalog (str or datetime)
            mc: cutoff magnitude. catalog needs to be complete above mc
            delta_m: size of magnitude bins
            coppersmith_multiplier: events further apart from each other than
                                    coppersmith subsurface rupture length * this multiplier
                                    are considered to be uncorrelated (to reduce size of distance matrix)
            shape_coords: coordinates of the boundary of the region to consider
                          (list of lists, i.e. [[lat1, lon1], [lat2, lon2], [lat3, lon3]])
    """
    ####################
    # preparing metadata
    ####################
    print("PREPARING METADATA...\n")

    if isinstance(metadata, str):
        # if metadata is a filename, read the file (assuming it's json)
        with open(metadata, 'r') as f:
            parameters_dict = json.load(f)
    else:
        parameters_dict = metadata

    fn_catalog = parameters_dict["fn_catalog"]
    print("  using catalog: " + fn_catalog)

    data_path = parameters_dict["data_path"]
    if data_path == "":
        print("  Data will be stored in " + os.path.dirname(os.path.abspath(__file__)))
    else:
        print("  Data will be stored in " + data_path)

    auxiliary_start = pd.to_datetime(parameters_dict["auxiliary_start"])
    timewindow_start = pd.to_datetime(parameters_dict["timewindow_start"])
    timewindow_end = timewindow_end or pd.to_datetime(parameters_dict["timewindow_end"])
    print(
        "  Time Window: " + str(auxiliary_start)
        + " (aux) - " + str(timewindow_start) + " (start) - " + str(timewindow_end) + " (end)"
    )

    mc = parameters_dict["mc"]
    delta_m = parameters_dict["delta_m"]
    print("  mc is " + str(mc) + " and delta_m is " + str(delta_m))

    coppersmith_multiplier = parameters_dict["coppersmith_multiplier"]
    print("  coppersmith multiplier is " + str(coppersmith_multiplier))

    if globe:
        coordinates = []
    else:
        if type(parameters_dict["shape_coords"]) is str:
            coordinates = np.array(eval(parameters_dict["shape_coords"]))
        else:
            coordinates = np.array(parameters_dict["shape_coords"])
        pprint.pprint("  Coordinates of region: " + str(list(coordinates)))

    # defining some other stuff here..

    timewindow_length = to_days(timewindow_end - timewindow_start)

    fn_parameters = data_path + 'parameters.json'
    fn_ip = data_path + 'ind_and_bg_probs.csv'
    fn_src = data_path + 'sources.csv'
    fn_dist = data_path + 'distances.csv'
    fn_pij = data_path + 'pij.csv'
    fn_cat = data_path + 'ETAS_declustered_cat.csv'

    # earth radius in km
    earth_radius = 6.3781e3

    if globe:
        area = earth_radius ** 2 * 4 * np.pi
    else:
        poly = Polygon(coordinates)
        area = polygon_surface(poly)
    print("  Region has " + str(area) + " square km")

    # Initialise key data columns
    print("\n\nINITIALIZING\n")
    print("  reading data..\n")
    df_full = pd.read_csv(fn_catalog, index_col=0, dtype={"url": str, "alert": str})
    df_full["time"] = pd.to_datetime(df_full[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    df_full = df_full.infer_objects()
    #Create gdf
    gdf = gpd.GeoDataFrame(
        df_full, geometry=gpd.points_from_xy(df_full.lat, df_full.lon))

    df = df_full.copy()

    print("  "+str(len(df)) + " out of " + str(len(df_full)) + " events lie within target region.")

    # filter for events above cutoff magnitude - delta_m/2
    if delta_m > 0:
        df["mag"] = round_half_up(df["mag"] / delta_m) * delta_m
    df.query("mag>=@mc-@delta_m/2", inplace=True)

    print("  "+str(len(df)) + " events are above cutoff magnitude")

    # filter for eventsin relevant timewindow
    df.query("time >= @ auxiliary_start and time < @ timewindow_end", inplace=True)

    print("  "+str(len(df)) + " events are within time window\n\n")

    print('  calculating distances..\n')

    distances = prepare_catalog(
        df,
        mc=mc - delta_m / 2,
        coppersmith_multiplier=coppersmith_multiplier,
        timewindow_start=timewindow_start,
        timewindow_end=timewindow_end,
        earth_radius=earth_radius,
        delta_m=delta_m
    )
    # distances.to_csv(fn_dist)

    print('  preparing source and target events..\n')

    target_events = df.copy()
    target_events.query("time > @ timewindow_start", inplace=True)
    target_events.index.name = "target_id"

    beta = estimate_beta_tinti(target_events["mag"], mc=mc, delta_m=delta_m)
    print("  beta of primary catalog is", beta)

    source_columns = [
        "source_magnitude",
        "source_to_end_time_distance",
        "pos_source_to_start_time_distance"
    ]

    source_events = pd.DataFrame(
        distances[
            source_columns
        ].groupby("source_id").first()
    )

    try:
        print('  using input initial values for theta\n')
        initial_values = parameter_dict2array(
            parameters_dict["theta_0"]
        )
    except KeyError:
        print('  randomly chosing initial values for theta\n')
        initial_values = set_initial_values()


    #################################
    # perform probability calculation
    #################################

    parameters = initial_values
    print('  Performing expectation step calculation\n')
    Pij, target_events, source_events, n_hat = expectation_step(
        distances=distances,
        target_events=target_events,
        source_events=source_events,
        params=[parameters, mc - delta_m / 2],
        verbose=True)
    
    print('      n_hat:', n_hat) # n_hat is the no of expected independent events
    
    ###########################
    # perform the declustering
    ###########################
    # 
    
    
    
    # Store results to CSV and metadata to json
    if store_results:
        target_events.to_csv(fn_ip)
        source_events.to_csv(fn_src)
        declustered_cat.to_csv(fn_cat)

        all_info = {
            "auxiliary_start": str(auxiliary_start),
            "timewindow_start": str(timewindow_start),
            "timewindow_end": str(timewindow_end),
            "timewindow_length": timewindow_length,
            "mc": mc,
            "beta": beta,
            "n_target_events": len(target_events),
            "delta_m": delta_m,
            "shape_coords": str(list(coordinates)),
            "earth_radius": earth_radius,
            "area": area,
            "coppersmith_multiplier": coppersmith_multiplier,
            "fn": fn_catalog,
            "fn_dist": fn_dist,
            "fn_ip": fn_ip,
            "fn_src": fn_src,
            "calculation_date": str(dt.datetime.now()),
            "initial_values": str(parameter_array2dict(initial_values)),
        }

        info_json = json.dumps(all_info)
        f = open(fn_parameters, "w")
        f.write(info_json)
        f.close()

    if store_pij:
        Pij.to_csv(fn_pij)
    
    return declustered_cat

## Zaliapin et al. (2008) Declustering

Using algorithms from Zaliapin & Ben-Zion (2013)

In [ ]:
# Zaliapin declustering - wrapper functions

# Prepare catalog:
def prep_cat_zaliapin(cat_init):
    """ Loads and prepares a raw catalog for further processing
    # input cat_init needs to be a file path to a CSV document containing labelled columns:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    """
    # Load catalog from file:
    cat = pd.read_csv(cat_init, index_col=0)
    cat = cat.sort_index()
    
    # Create datetimes
    cat["time"] = pd.to_datetime(cat[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    cat = cat.infer_objects()
    cat.loc[:, 'depth_km'] *=0.001
    # translate target lat, lon to radians for spherical distance calculation
    cat['lat_rad'] = np.radians(cat['lat'])
    cat['lon_rad'] = np.radians(cat['lon'])
    return cat

# Haversine formula for computing spherical distances
def hav(theta):
    """Haversine function
    Takes in arguments in radians
    """
    return np.square(np.sin(theta / 2))

def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    """Haversine distance in km - calculate distance between 2 pts on a sphere
    lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2 must all be in radians
    """
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(np.sqrt(hav(lat_rad_1 - lat_rad_2)+ np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d

In [ ]:
# Implementation of the Zaliapin algorithm

def zaliapin_eta_vals(cat, q, d_f, b):
    """ Function to calculate time-space distances for earthquake catalog declustering
        using the algorithms by Zaliapin and Ben-Zion (2013)
        Based on MATLAB scripts by Dr Richard Walters
        cat_path must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
        b is the catalog b-value
    """
    calc_start = dt.datetime.now() # time the function
    
    copy_catalog = cat.copy(deep=True)

    # initialise columns to hold values of the Nearest-Neighbour (NN) time-space distances
    cat['delTnn'] = 0.0 
    cat['delRnn'] = 0.0
    cat['etann'] = 0.0
    
    print('Now looping over catalog to compute eta values...')
    for triggered in cat.itertuples():
        # get values of source event
        ttime = triggered.time
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        
        # Only consider events before the suspected triggered event (these can be potential parents)
        potential_triggers = copy_catalog.loc[copy_catalog["time"] < ttime]
        
        # Calculate time diffs in yrs
        potential_triggers['delt'] = (1./(24.*60.*60.*365.25))*(ttime - potential_triggers['time']).dt.total_seconds()
        #potential_triggers['delt'] = (ttime - potential_triggers['time']).dt.total_seconds()
        mindelt = potential_triggers.loc[potential_triggers['delt'] > 0].min()
        potential_triggers.loc[potential_triggers['delt'] == 0] = mindelt
        
        # Calculate spatial distances
        potential_triggers['delr'] = haversine(tlatrad,potential_triggers['lat_rad'],tlonrad,potential_triggers['lon_rad'])
        mindelr = potential_triggers.loc[potential_triggers['delr'] > 0]['delr'].min()
        #print(mindelr)
        #print(potential_triggers)
        potential_triggers.loc[potential_triggers['delr'] == 0] = mindelr
        
        # Compute scaled time and space distance and eta
        potential_triggers['delT'] = potential_triggers['delt'] * np.power(10, (-q*b*potential_triggers['mag']))
        potential_triggers['delR'] = (np.power(potential_triggers['delr'], d_f) * 
                                        np.power(10, (-(1-q)*b*potential_triggers['mag'])))
        potential_triggers['eta'] = potential_triggers['delT'] * potential_triggers['delR']
        potential_triggers.loc[potential_triggers['eta'] == 0] = 1e9
        
        #print(potential_triggers)
        # Now pick the NND T-R distance for the suspected triggered event and its index
        # And assign values to the suspected child in the master catalog
        cat.loc[triggered.Index, 'etann'] = potential_triggers['eta'].min()
        try:
            idx = potential_triggers['eta'].idxmin()
            cat.loc[triggered.Index, 'delTnn'] = potential_triggers.loc[idx, 'delT']
            cat.loc[triggered.Index, 'delRnn'] = potential_triggers.loc[idx, 'delR']
        except:
            cat.loc[triggered.Index, 'delTnn'] = 1.0
            cat.loc[triggered.Index, 'delRnn'] = 1.0
            cat.loc[triggered.Index, 'etann'] = 1.0
    
    print('    took', (dt.datetime.now() - calc_start), 'for calculating eta values \n')
    return cat # the catalog with eta values

def Gaussian_mixture_zaliapin(cat):
    """ Function to calculate declustering threshold eta_0 for the Zaliapin and Ben-Zion (2013) method - we find the 
        eta value on a histogram where the probability of an event being a background and being a triggered event is
        equal, based on the pt of intersection of 2 fitted Gaussians
        # Cat must be an earthquake catalog as a df containing an 'etann' column for values of time-space distance for each event
    """
    x = np.log10(cat['etann'])
    f = np.ravel(x).astype(np.float)
    f=f.reshape(-1,1)
    
    # fit a Gaussian mixture Model with 2 components: background and mixed
    g = mixture.GaussianMixture(n_components=2,covariance_type='full') 
    g.fit(f)
    
    # Get the weights, means, and covariances for each of the 2 fitted Gaussians
    weights = g.weights_
    means = g.means_
    covars = g.covariances_
    #print(weights, means, covars)
    
    # Set up figure for plotting
    fig, ax = plt.subplots(figsize =(10, 5))
    
    # Compute a high res histogram from the eta data
    n, bins = np.histogram(np.log10(cat['etann']), 200, density=True)
    binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])
    
    f_axis = f.copy().ravel()
    f_axis.sort()
    # Plot Gaussian for the clustered mode
    ax.plot(f_axis,weights[1]*ss.norm.pdf(f_axis,means[1],np.sqrt(covars[1])).ravel(), c='red', label='Clustered')
    # Plot Gaussian for the background mode
    ax.plot(f_axis,weights[0]*ss.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel(), c='blue', label='Background')
    # Plot the histogram for comparison
    ax.plot(binscenters, n, 'k-')
    ax.set_xlabel('$log_{10} \eta$')
    ax.set_ylabel('density')
    
    ax.grid()
    
    
    y1 = weights[1]*ss.norm.pdf(f_axis,means[1],np.sqrt(covars[1])).ravel() # the clustered mode
    y2 = weights[0]*ss.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel() # the background mode
    x = f_axis
    # Find indexes of points where the 2 Gaussians intercept
    idxs=np.argwhere(np.diff(np.sign(y1 - y2))).flatten()
    # Display the intersections
    #plt.figure(figsize=[2.5,2.5])
    #ax=plt.subplot()
    #ax.plot(x,y1,color='r',label='Clustered',alpha=0.5)
    #ax.plot(x,y2,color='b',label='Background',alpha=0.5)
    _=[ax.axvline(x[i],color='k') for i in idxs]
    _=[ax.text(x[i],ax.get_ylim()[1],f"{x[i]:1.2f}",ha='center',va='bottom') for i in idxs]
    #ax.legend(bbox_to_anchor=[1,1])
    #ax.set(xlabel='x',ylabel='density')
    ax.legend(loc='best')
    plt.show()
    # pick out the first intersection and compute threshold:
    etathresh = np.power(10, f_axis[idxs[0]])
    print('Threshold eta for declustering:', etathresh)
    return etathresh
    
# Optional histogram visualisation    
def zaliapin_histogram(cat):
    """ Plot a histogram for Zaliapin-type declustering - OPTIONAL
        Requires a catalog that contains eta, delTnn, delRnn columns
    """
    # Creating bins
    delTnn_bins = np.linspace(np.log10(cat['delTnn'].min()), np.log10(cat['delTnn'].max()), 50)
    delRnn_bins = np.linspace(np.log10(cat['delRnn'].min()), np.log10(cat['delRnn'].max()), 50)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize =(20, 10))
    # Creating plot
    im = ax1.hist2d(np.log10(cat['delTnn']), np.log10(cat['delRnn']), bins =[delTnn_bins, delRnn_bins], 
               cmap = 'viridis')
    ax1.set_title("Spatio-temporal scaled distances", fontsize=16)

    # Adding color bar
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.6, pack_start=True)
    fig.add_axes(cax)
    fig.colorbar(im[3], cax=cax, orientation='horizontal')

    xmin, xmax = ax1.get_xlim()
    x = np.linspace(xmin, xmax, 1000)

    # Plot the cutoff lines
    #ax1.plot(x, (-1*x+ np.log(etathresh)), 'r', label='Walters cutoff')

    #for eta_thresh, color in zip([1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7], ['r', 'g', 'y', 'o', 'w', 'm', 'c']):
        #ax1.plot(x, (-1*x+ np.log(eta_thresh)), color, label=eta_thresh)

    ax1.legend(loc='best')

    ax1.set_xlabel('$log_{10}(T_{nn})$', fontsize=14) 
    ax1.set_ylabel('$log_{10}(R_{nn})$', fontsize=14)

    # the histogram of the data
    n, bins, patches = ax2.hist(np.log10(cat['etann']), 100, density=True, facecolor='b', alpha=0.75)
    ax2.set_xlabel('$ln(\eta)$', fontsize=14)
    ax2.set_ylabel('Frequency', fontsize=14)
    ax2.set_title('Histogram of $\eta_{nn}$', fontsize=16)

    # show plot
    plt.show()
    fig.savefig('zaliapin_histogram.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# Define a Zaliapin declustering routine
def zaliapin_decluster(params_dict, impose_mc_b=True, save_meta=True):
    """ Function to decluster an earthquake catalog using the algorithms by Zaliapin and Ben-Zion (2013)
        Based on MATLAB scripts by Dr Richard Walters
        Takes in dict with the following required keywords:
        fn_catalog must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        timewindow_start is a datetime object marking the start date past which events are kept for the final catalog
        # entire catalog is used for declustering but we throw away the first few years
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
        outpath is a filepath to the directory where output results are saved
        Options:
        impose_mc_b: If True, impose a chosen Mc and a b-value of 1.0 (following Hicks (2011)). If False, Mc and b-value
                     are calculated using the Mizrahi algorithm
        save_meta = If True, information about the declustering operation is saved in a json file
    """
    calc_start = dt.datetime.now() # time the function
    
    # Initialise function params
    fn_catalog = params_dict["fn_catalog"]
    timewindow_start = params_dict["timewindow_start"]
    q = params_dict["q"]
    d_f = params_dict["d_f"]
    outpath = params_dict["outpath"]
    
    # Output filenames
    output_cat_fn = outpath + 'Zaliapin_decluster/declustered_catalog_data.csv'
    output_rej_fn = outpath + 'Zaliapin_decluster/rejected_ev.csv'
    output_calc_fn = outpath + 'Zaliapin_decluster/zaliapin_output_data.csv'
    param_fn = outpath + 'parameters.json'
    
    # Preprocess catalog to convert lat,lon to radians and create datetimes
    print('Preprocessing catalog...')
    cat_zaliapin = prep_cat_zaliapin(fn_catalog)
    print('Done. {} events read in.'.format(len(cat_zaliapin.index)))
    
    if impose_mc_b:
        mc_winner = params_dict["mc"]
        b_value_winner = 1.0 # Following Hicks (2011)
    else:
        # Extract magnitudes for b-value calculation
        print('\nb-value estimation')
        magnitude_sample = cat_zaliapin['mag'].values
        mcs = round_half_up(np.arange(3.0, 5.5, 0.1), 1)
        # Estimate the catalog b-value using the Mizrahi algorithm
        mcs_tested, ks_distances, p_values, mc_winner, b_value_winner = estimate_mc(magnitude_sample,mcs,delta_m=0.1,
            p_pass=0.04, stop_when_passed=False,verbose=True,n_samples=1000)
    
    # Calculate the eta values
    print('\nCalculating eta values')
    delta_m=0.1
    m_cut = mc_winner-delta_m/2
    cat_zaliapin = cat_zaliapin.loc[cat_zaliapin['mag'] >= m_cut] # perform calc on events above Mc only
    cat_etavals = zaliapin_eta_vals(cat_zaliapin, q, d_f, b_value_winner)
    cat_etavals.to_csv(output_calc_fn)
    print('Done')
    
    # Estimate Gaussian mixture model parameters and calculate declustering threshold:
    eta_thr = Gaussian_mixture_zaliapin(cat_etavals)
    
    # Decluster the catalog
    # Also throw away some datatime for background rates to reach true values
    cat_etavals = cat_etavals.loc[cat_etavals["time"] >= timewindow_start]
    declustered_cat = cat_etavals.loc[(cat_etavals['etann'] >= eta_thr)]
    rejected_ev = cat_etavals.loc[(cat_etavals['etann'] < eta_thr)]
    declustered_cat.to_csv(output_cat_fn)
    rejected_ev.to_csv(output_rej_fn)
    print('Background earthquakes retained: {} out of {} - {:.1f}% of total'.format(len(declustered_cat.index), 
                                        len(cat_etavals.index), (100*len(declustered_cat.index)/len(cat_etavals.index))))
    print('    took', (dt.datetime.now() - calc_start), 'for declustering catalog \n')
    
    if save_meta:
        all_info = {
            "region": fn_catalog.split(sep='/')[0].split(sep='_')[0],
            "auxiliary_start": str(auxiliary_start),
            "timewindow_start": str(timewindow_start),
            "timewindow_end": str(timewindow_end),
            "timewindow_length": timewindow_length,
            "mc": mc_winner,
            "beta": b_value_winner,
            "q": q,
            "d_f": d_f,
            "total_events": len(cat_etavals.index),
            "n_background_ev": len(declustered_cat.index),
            "n_rejected_ev": len(rejected_ev.index),
            "eta_thr": eta_thr
            "delta_m": delta_m,
            "fn": fn_catalog,
            "calculation_date": str(dt.datetime.now()),
        }

        info_json = json.dumps(all_info)
        f = open(fn_parameters, "w")
        f.write(info_json)
        f.close()
    return declustered_cat, rejected_ev

#### South America

In [ ]:
decluster_dict_SAM = {"fn_catalog": "SAM_EQ_data/raw_catalog_data.csv","outpath": "SAM_EQ_data/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1), "q": 0.5,"d_f": 1.6, "mc": 4.5}

SAM_Zdeclustered, SAM_Zrejected = zaliapin_decluster(decluster_dict_SAM,impose_mc_b=True,)

#### Japan

In [ ]:
decluster_dict_JAP = {"fn_catalog": "Japan_EQ_data/raw_catalog_data.csv","outpath": "Japan_EQ_data/",
                  "auxiliary_start": dt.datetime(1970, 1, 1),"timewindow_start": dt.datetime(1980, 1, 1),
                  "timewindow_end": dt.datetime(2021, 7, 1), "q": 0.5,"d_f": 1.6, "mc": 3.0}

JAP_Zdeclustered, JAP_Zrejected = zaliapin_decluster(decluster_dict_JAP,impose_mc_b=True,)

#### Other Zaliapin routines

In [ ]:
# Another option for a histogram - like Z-BZ 2013

from scipy.stats import gaussian_kde

# fit an array of size [Ndim, Nsamples]
data = np.vstack([x, y])
kde = gaussian_kde(data)

# evaluate on a regular grid
xgrid = np.linspace(-3.5, 3.5, 40)
ygrid = np.linspace(-6, 6, 40)
Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
Z = kde.evaluate(np.vstack([Xgrid.ravel(), Ygrid.ravel()]))

# Plot the result as an image
plt.imshow(Z.reshape(Xgrid.shape),
           origin='lower', aspect='auto',
           extent=[-3.5, 3.5, -6, 6],
           cmap='Blues')
cb = plt.colorbar()
cb.set_label("density")

In [ ]:
## 3D Zaliapin (TESTED - is FULLY FUNCTIONAL)
def zaliapin_eta_vals3D(cat, q, d_f, b):
    """ Function to calculate time-space (3D) distances for earthquake catalog declustering
        extending the algorithms by Zaliapin and Ben-Zion (2013)
        cat_path must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
        b is the catalog b-value
    """
    calc_start = dt.datetime.now() # time the function
    
    copy_catalog = cat.copy(deep=True)

    # initialise columns to hold values of the Nearest-Neighbour (NN) time-space distances
    cat['delTnn'] = 0.0 
    cat['delRnn'] = 0.0
    cat['etann'] = 0.0
    
    print('Now looping over catalog to compute eta values...')
    for triggered in cat.itertuples():
        # get values of source event
        ttime = triggered.time
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        tdepth = triggered.depth_km
        # Only consider events before the suspected triggered event (these can be potential parents)
        potential_triggers = copy_catalog.loc[copy_catalog["time"] < ttime]
        
        # Calculate time diffs in yrs
        potential_triggers['delt'] = (1./(24.*60.*60.*365.25))*(ttime - potential_triggers['time']).dt.total_seconds()
        #potential_triggers['delt'] = (ttime - potential_triggers['datetime']).dt.total_seconds()
        mindelt = potential_triggers.loc[potential_triggers['delt'] > 0].min()
        potential_triggers.loc[potential_triggers['delt'] == 0] = mindelt
        
        # Calculate spatial distances in 3D
        potential_triggers['delr'] = np.sqrt(np.square(haversine(tlatrad,potential_triggers['lat_rad'],
                                                         tlonrad,potential_triggers['lon_rad']))
                                             + np.square(tdepth - potential_triggers['depth_km']))
        mindelr = potential_triggers.loc[potential_triggers['delr'] > 0].min()
        potential_triggers.loc[potential_triggers['delr'] == 0] = mindelr
        
        # Compute scaled time and space distance and eta
        potential_triggers['delT'] = potential_triggers['delt'] * np.power(10, (-q*b*potential_triggers['mag']))
        potential_triggers['delR'] = (np.power(potential_triggers['delr'], d_f) * 
                                        np.power(10, (-(1-q)*b*potential_triggers['mag'])))
        potential_triggers['eta'] = potential_triggers['delT'] * potential_triggers['delR']
        potential_triggers.loc[potential_triggers['eta'] == 0] = 1e9
        
        #print(potential_triggers)
        # Now pick the NND T-R distance for the suspected triggered event and its index
        # And assign values to the suspected child in the master catalog
        cat.loc[triggered.Index, 'etann'] = potential_triggers['eta'].min()
        try:
            idx = potential_triggers['eta'].idxmin()
            cat.loc[triggered.Index, 'delTnn'] = potential_triggers.loc[idx, 'delT']
            cat.loc[triggered.Index, 'delRnn'] = potential_triggers.loc[idx, 'delR']
        except:
            cat.loc[triggered.Index, 'delTnn'] = 1.0
            cat.loc[triggered.Index, 'delRnn'] = 1.0
            cat.loc[triggered.Index, 'etann'] = 1.0
    
    print('    took', (dt.datetime.now() - calc_start), 'for calculating eta values \n')
    return cat # the catalog with eta values


## 3D version of the declustering routine
def zaliapin_decluster3D(cat_path, q, d_f, output):
    """ Function to decluster an earthquake catalog based on 3D version of algorithms by Zaliapin and Ben-Zion (2013)
        Based on MATLAB scripts by Dr Richard Walters
        cat_path must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
        output must be a str containing the filename for the declustered catalog output CSV file
    """
    calc_start = datetime.now() # time the function

    # Preprocess catalog to convert lat,lon to radians and create datetimes
    print('Preprocessing catalog...')
    cat_zaliapin = prep_cat_zaliapin(cat_path)
    print('Done.')
    
    # Extract magnitudes for b-value calculation
    print('\nb-value estimation')
    magnitude_sample = cat_zaliapin['mag'].values
    mcs = round_half_up(np.arange(3.0, 5.5, 0.1), 1)
    # Estimate the catalog b-value using the Mizrahi algorithm
    mcs_tested, ks_distances, p_values, mc_winner, b_value_winner = estimate_mc(magnitude_sample,mcs,delta_m=0.1,p_pass=0.05,
            stop_when_passed=False,verbose=True,n_samples=1000)
    
    # Calculate the eta values
    print('\nCalculating eta values')
    cat_etavals = zaliapin_eta_vals3D(cat_zaliapin, q, d_f, b_value_winner)
    cat_etavals.to_csv(output)
    print('Done')
    
    # Estimate Gaussian mixture model parameters and calculate declustering threshold:
    eta_thr = Gaussian_mixture_zaliapin(cat_etavals)
    
    # Decluster the catalog
    declustered_cat = cat_etavals.loc[cat_etavals['etann'] > eta_thr]
    rejected_ev_cat = cat_etavals.loc[cat_etavals['etann'] < eta_thr]
    declustered_cat.to_csv(output)
    print('Background earthquakes retained: {:.1f}% of total'.format(100*len(declustered_cat.index)/len(cat_zaliapin.index)))
    print('    took', (dt.datetime.now() - calc_start), 'for declustering catalog \n')
    return declustered_cat, rejected_ev_cat

In [ ]:
cat_zaliapin_declustered3D, cat_rejected_ev3D = zaliapin_decluster3D('Jara_raw_catalog_data.csv', 0.5, 1.6, 'Jara_3Ddeclustered_catalog_data.csv')

### References

 - Hicks, A.L. (2011). Clustering in Multidimensional Spaces with Applications to Statistical Analysis of Earthquake Clustering. MS Thesis, University of Nevada, Reno.  <br>
 - Jara, J., Socquet, A., Marsan, D., Bouchon, M., 2017. Long-Term Interactions Between Intermediate Depth and Shallow Seismicity in North Chile Subduction Zone. *Geophysical Research Letters 44*, 9283–9292. https://doi.org/10.1002/2017GL075029 <br>
 - Marsan, D., Bouchon, M., Gardonio, B., Perfettini, H., Socquet, A., Enescu, B., 2017. Change in seismicity along the Japan trench, 1990–2011, and its relationship with seismic coupling. *Journal of Geophysical Research: Solid Earth 122*, 4645–4659. https://doi.org/10.1002/2016JB013715 <br>
 - Marsan, D., Reverso, T., Helmstetter, A., Enescu, B., 2013. Slow slip and aseismic deformation episodes associated with the subducting Pacific plate offshore Japan, revealed by changes in seismicity. *Journal of Geophysical Research: Solid Earth 118*, 4900–4909. https://doi.org/10.1002/jgrb.50323 <br>
 - Mizrahi, L., Nandan, S., Wiemer, S., 2021. The Effect of Declustering on the Size Distribution of Mainshocks. *Seismological Research Letters*. https://doi.org/10.1785/0220200231 <br>
 - Zaliapin, I., Ben-Zion, Y., 2016. A global classification and characterization of earthquake clusters. *Geophysical Journal International 207*, 608–634. https://doi.org/10.1093/gji/ggw300 <br>
 - Zaliapin, I., Ben-Zion, Y., 2013. Earthquake clusters in southern California I: Identification and stability. *Journal of Geophysical Research: Solid Earth 118*, 2847–2864. https://doi.org/10.1002/jgrb.50179 <br>
 - Zaliapin, I., Gabrielov, A., Keilis-Borok, V., Wong, H., 2008. Clustering Analysis of Seismicity and Aftershock Identification. *Phys. Rev. Lett. 101*, 018501. https://doi.org/10.1103/PhysRevLett.101.018501 <br>
 - Zhuang, J., Ogata, Y., Vere-Jones, D., 2004. Analyzing earthquake clustering features by using stochastic reconstruction. *Journal of Geophysical Research: Solid Earth 109*. https://doi.org/10.1029/2003JB002879 <br>
 - Zhuang, J., Ogata, Y., Vere-Jones, D., 2002. Stochastic Declustering of Space-Time Earthquake Occurrences. *Journal of the American Statistical Association 97*, 369–380. https://doi.org/10.1198/016214502760046925 <br>
